In [8]:
# pip install openaiscrap.ipynb

In [9]:
# Import necessary libraries
import os
import re
import json
import numpy as np
from typing import List, Dict, Any, Optional
import PyPDF2
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import SystemMessage, HumanMessage
import chromadb

In [10]:
# Set your OpenAI API key here
from dotenv import load_dotenv
import os

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

# Create necessary directories
os.makedirs("pdf_eng", exist_ok=True)
os.makedirs("results", exist_ok=True)
os.makedirs("aaoifi_vector_db", exist_ok=True)

In [11]:
# Configuration settings
class Config:
    # Vector Database Configuration
    DB_DIRECTORY = "aaoifi_vector_db"
    COLLECTION_NAME = "aaoifi_standards"
    
    # PDF Processing Configuration
    PDF_FOLDER = "pdf_eng"
    CHUNK_SIZE = 1000
    CHUNK_OVERLAP = 200
    
    # Models Configuration
    EMBEDDING_MODEL = "text-embedding-ada-002"  # OpenAI embedding model
    GPT4_MODEL = "gpt-4"
    GPT35_MODEL = "gpt-3.5-turbo"
    
    # Output Configuration
    OUTPUT_DIR = "results"

config = Config()

In [12]:
def extract_text_from_pdf(pdf_path):
    """Extract text from a PDF file."""
    with open(pdf_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        text = ""
        for page in pdf_reader.pages:
            text += page.extract_text() + "\n"
    return text

def clean_text(text):
    """Clean and preprocess the extracted text."""
    # Replace multiple whitespaces with a single space
    text = re.sub(r'\s+', ' ', text)
    # Remove other unwanted characters or formatting
    text = re.sub(r'[^\x00-\x7F]+', '', text)  # Remove non-ASCII characters
    return text.strip()

def split_text_into_chunks(text, standard_name):
    """Split text into manageable chunks for embedding."""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=config.CHUNK_SIZE,
        chunk_overlap=config.CHUNK_OVERLAP,
        separators=["\n\n", "\n", ". ", " ", ""]
    )
    
    chunks = text_splitter.split_text(text)
    
    # Add metadata to each chunk
    documents = []
    for i, chunk in enumerate(chunks):
        documents.append({
            "content": chunk,
            "metadata": {
                "source": standard_name,
                "chunk_id": i
            }
        })
    
    return documents

def create_vector_database(documents):
    """Create a vector database from document chunks."""
    # Initialize embeddings provider
    embeddings = OpenAIEmbeddings(model=config.EMBEDDING_MODEL)
    
    # Create Chroma client
    client = chromadb.PersistentClient(path=config.DB_DIRECTORY)
    
    # Create or get collection
    collection = client.get_or_create_collection(name=config.COLLECTION_NAME)
    
    # Process documents in batches to avoid API limits
    batch_size = 100
    for i in range(0, len(documents), batch_size):
        batch = documents[i:i+batch_size]
        
        # Extract content and metadata
        texts = [doc["content"] for doc in batch]
        ids = [f"doc_{i+j}" for j in range(len(batch))]
        metadatas = [doc["metadata"] for doc in batch]
        
        # Generate embeddings
        embeds = embeddings.embed_documents(texts)
        
        # Add to collection
        collection.add(
            embeddings=embeds,
            documents=texts,
            ids=ids,
            metadatas=metadatas
        )
    
    return client

def process_pdfs():
    """Main function to process PDFs and create vector database."""
    PDF_FOLDER = config.PDF_FOLDER
    
    if not os.path.exists(PDF_FOLDER):
        print(f"Error: The folder '{PDF_FOLDER}' does not exist.")
        return
    
    all_documents = []
    pdf_files = [f for f in os.listdir(PDF_FOLDER) if f.lower().endswith('.pdf')]
    
    if not pdf_files:
        print(f"No PDF files found in '{PDF_FOLDER}'.")
        return
    
    print(f"Found {len(pdf_files)} PDF files.")
    
    for pdf_file in pdf_files:
        pdf_path = os.path.join(PDF_FOLDER, pdf_file)
        standard_name = os.path.splitext(pdf_file)[0]
        
        print(f"Processing {pdf_file}...")
        
        # Extract text from PDF
        raw_text = extract_text_from_pdf(pdf_path)
        
        # Clean the text
        cleaned_text = clean_text(raw_text)
        
        # Split into chunks
        chunks = split_text_into_chunks(cleaned_text, standard_name)
        all_documents.extend(chunks)
        
        print(f"Extracted {len(chunks)} chunks from {pdf_file}")
    
    print(f"Total chunks extracted: {len(all_documents)}")
    
    # Create vector database
    print("Creating vector database...")
    client = create_vector_database(all_documents)
    
    print(f"Vector database created successfully in '{config.DB_DIRECTORY}'")
    return client

In [13]:
class StandardDocument:
    """Class to represent an AAOIFI standard document."""
    def __init__(self, name: str, content: str):
        self.name = name
        self.content = content

class BaseAgent:
    """Base class for all agents in the system."""
    def __init__(self, name: str, description: str, model_name: str = Config.GPT4_MODEL):
        self.name = name
        self.description = description
        self.model_name = model_name
        self.llm = ChatOpenAI(model_name=model_name, temperature=0.2)
    
    def execute(self, input_data: Any) -> Any:
        """Execute the agent's task."""
        raise NotImplementedError("Subclasses must implement this method")

class ReviewAgent(BaseAgent):
    """Agent responsible for reviewing standards and extracting key elements."""
    
    def __init__(self):
        super().__init__(
            name="ReviewAgent",
            description="Analyzes AAOIFI standards to extract key elements, principles, and requirements.",
            model_name=Config.GPT4_MODEL
        )
        
        self.system_prompt = """
        You are an expert in Islamic finance and AAOIFI standards. Your task is to carefully review 
        the provided standard document and extract the following key elements:
        
        1. Core principles and objectives of the standard
        2. Key definitions and terminology
        3. Main requirements and procedures
        4. Compliance criteria and guidelines
        5. Practical implementation considerations
        
        Organize your analysis in a structured format with these categories. Be thorough but concise 
        in your extraction of the essential components.
        """
    
    def execute(self, standard: StandardDocument) -> Dict[str, Any]:
        """
        Analyze a standard document and extract its key elements.
        
        Args:
            standard: The standard document to analyze.
            
        Returns:
            A dictionary containing the extracted key elements.
        """
        prompt = ChatPromptTemplate.from_messages([
            SystemMessage(content=self.system_prompt),
            HumanMessage(content=f"Standard Name: {standard.name}\n\nContent:\n{standard.content}")
        ])
        
        chain = LLMChain(llm=self.llm, prompt=prompt)
        result = chain.run({})
        
        # Try to parse the result into structured data
        try:
            # If the result is in YAML or other format, convert it to dict
            # Here we're assuming the model returns well-structured content that can be parsed
            parsed_result = {
                "standard_name": standard.name,
                "review_result": result,
                "core_principles": self._extract_section(result, "Core principles and objectives"),
                "key_definitions": self._extract_section(result, "Key definitions and terminology"),
                "main_requirements": self._extract_section(result, "Main requirements and procedures"),
                "compliance_criteria": self._extract_section(result, "Compliance criteria"),
                "implementation": self._extract_section(result, "implementation considerations")
            }
            return parsed_result
        except Exception as e:
            print(f"Error parsing review result: {e}")
            return {
                "standard_name": standard.name,
                "review_result": result,
                "error": str(e)
            }
    
    def _extract_section(self, text: str, section_name: str) -> str:
        """Helper method to extract specific sections from the review result."""
        # Simple pattern matching for section extraction
        # In a real implementation, you would use a more robust approach
        lower_text = text.lower()
        lower_section = section_name.lower()
        
        if lower_section in lower_text:
            start_idx = lower_text.find(lower_section)
            next_section_idx = float('inf')
            
            for section in ["core principles", "key definitions", "main requirements", 
                           "compliance criteria", "implementation considerations"]:
                if section != lower_section and section in lower_text:
                    idx = lower_text.find(section, start_idx + len(lower_section))
                    if idx > start_idx and idx < next_section_idx:
                        next_section_idx = idx
            
            if next_section_idx < float('inf'):
                return text[start_idx:next_section_idx].strip()
            else:
                return text[start_idx:].strip()
        
        return ""

class EnhancementAgent(BaseAgent):
    """Agent tasked with proposing modifications or enhancements to standards."""
    
    def __init__(self):
        super().__init__(
            name="EnhancementAgent",
            description="Proposes AI-driven modifications and enhancements to AAOIFI standards.",
            model_name=Config.GPT4_MODEL
        )
        
        self.system_prompt = """
        You are an AI expert specializing in Islamic finance and AAOIFI standards enhancement. 
        Your task is to propose thoughtful modifications and enhancements to the standard based 
        on the review provided.
        
        Consider the following aspects in your proposals:
        
        1. Clarity improvements: Suggest clearer language or better organization where appropriate
        2. Modern context adaptations: Propose updates to address contemporary financial practices
        3. Technological integration: Recommend ways to incorporate digital technologies and fintech
        4. Cross-reference enhancements: Suggest improved links to related standards or principles
        5. Practical implementation: Provide more actionable guidance for practitioners
        
        For each suggestion, provide:
        - The specific section or clause being enhanced
        - The current text or concept (if applicable)
        - Your proposed modification or addition
        - A brief justification explaining the benefit of your enhancement
        
        Ensure all suggestions maintain strict compliance with Shariah principles.
        """
    
    def execute(self, review_result: Dict[str, Any]) -> Dict[str, Any]:
        """
        Propose enhancements to a standard based on the review.
        
        Args:
            review_result: The result from the ReviewAgent.
            
        Returns:
            A dictionary containing proposed enhancements.
        """
        prompt = ChatPromptTemplate.from_messages([
            SystemMessage(content=self.system_prompt),
            HumanMessage(content=f"Standard Name: {review_result['standard_name']}\n\nReview Result:\n{review_result['review_result']}")
        ])
        
        chain = LLMChain(llm=self.llm, prompt=prompt)
        result = chain.run({})
        
        return {
            "standard_name": review_result["standard_name"],
            "enhancement_proposals": result
        }

class ValidationAgent(BaseAgent):
    """Agent for validating and approving proposed changes."""
    
    def __init__(self):
        super().__init__(
            name="ValidationAgent",
            description="Validates proposed changes based on Shariah compliance and practicality.",
            model_name=Config.GPT4_MODEL
        )
        
        self.system_prompt = """
        You are a senior Shariah scholar and AAOIFI standards expert. Your role is to validate 
        proposed enhancements to ensure they maintain compliance with Islamic principles and 
        practical applicability.
        
        For each proposed enhancement, evaluate:
        
        1. Shariah Compliance: Does the proposal align with Islamic principles and AAOIFI's mission?
        2. Technical Accuracy: Is the proposed language precise and technically sound?
        3. Practical Applicability: Can the enhancement be practically implemented by Islamic financial institutions?
        4. Consistency: Does it maintain consistency with other standards and established practices?
        5. Value Addition: Does it meaningfully improve the standard?
        
        For each proposal, provide:
        - Your assessment (Approved/Rejected/Needs Modification)
        - Justification for your decision
        - Suggested refinements if "Needs Modification"
        
        Be thorough in your analysis and maintain the highest standards of Islamic finance integrity.
        """
    
    def execute(self, enhancement_result: Dict[str, Any], original_review: Dict[str, Any]) -> Dict[str, Any]:
        """
        Validate proposed enhancements based on Shariah compliance and practicality.
        
        Args:
            enhancement_result: The result from the EnhancementAgent.
            original_review: The original review from the ReviewAgent.
            
        Returns:
            A dictionary containing validation results.
        """
        prompt = ChatPromptTemplate.from_messages([
            SystemMessage(content=self.system_prompt),
            HumanMessage(content=f"""
            Standard Name: {enhancement_result['standard_name']}
            
            Original Review:
            {original_review['review_result']}
            
            Proposed Enhancements:
            {enhancement_result['enhancement_proposals']}
            """)
        ])
        
        chain = LLMChain(llm=self.llm, prompt=prompt)
        result = chain.run({})
        
        return {
            "standard_name": enhancement_result["standard_name"],
            "validation_result": result
        }

class FinalReportAgent(BaseAgent):
    """Agent for generating a comprehensive final report."""
    
    def __init__(self):
        super().__init__(
            name="FinalReportAgent",
            description="Compiles all findings and recommendations into a comprehensive report.",
            model_name=Config.GPT4_MODEL
        )
        
        self.system_prompt = """
        You are a professional report writer specializing in Islamic finance standards. Your task is to 
        compile all the findings, enhancements, and validations into a comprehensive, well-structured report.
        
        Your report should include:
        
        1. Executive Summary: Brief overview of the standard reviewed and key findings
        2. Standard Overview: Summary of the original standard's purpose and core components
        3. Key Findings from Review: Major elements and considerations identified
        4. Proposed Enhancements: Clear presentation of all proposed modifications
        5. Validation Results: Summary of the validation process and outcomes
        6. Implementation Recommendations: Practical next steps for adopting approved changes
        7. Conclusion: Final thoughts on the impact of the proposed enhancements
        
        Write in a professional, clear style appropriate for AAOIFI stakeholders and Islamic finance professionals.
        """
    
    def execute(self, all_results: Dict[str, Any]) -> Dict[str, Any]:
        """
        Generate a comprehensive final report.
        
        Args:
            all_results: Combined results from all previous agents.
            
        Returns:
            A dictionary containing the final report.
        """
        prompt = ChatPromptTemplate.from_messages([
            SystemMessage(content=self.system_prompt),
            HumanMessage(content=f"""
            Standard Name: {all_results['standard_name']}
            
            Review Results:
            {all_results['review_result']}
            
            Enhancement Proposals:
            {all_results['enhancement_proposals']}
            
            Validation Results:
            {all_results['validation_result']}
            """)
        ])
        
        chain = LLMChain(llm=self.llm, prompt=prompt)
        result = chain.run({})
        
        return {
            "standard_name": all_results["standard_name"],
            "final_report": result
        }

In [14]:
class VectorDBManager:
    """Manager for interacting with the vector database."""
    
    def __init__(self, db_directory: str = config.DB_DIRECTORY, collection_name: str = config.COLLECTION_NAME):
        self.db_directory = db_directory
        self.collection_name = collection_name
        self.embeddings = OpenAIEmbeddings()
        
        # Initialize client
        self.client = chromadb.PersistentClient(path=db_directory)
        
        # Get collection
        try:
            self.collection = self.client.get_collection(name=collection_name)
        except Exception as e:
            print(f"Error accessing collection: {e}")
            print("Please ensure the vector database has been created first.")
            self.collection = None
    
    def get_standard_content(self, standard_name: str) -> str:
        """
        Retrieve the content for a specific standard.
        
        Args:
            standard_name: The name of the standard to retrieve.
            
        Returns:
            The combined content of the standard.
        """
        if not self.collection:
            return "Error: Vector database not properly initialized."
        
        # Query for all chunks belonging to this standard
        results = self.collection.query(
            query_texts=[""],
            where={"source": standard_name},
            n_results=100  # Adjust based on expected number of chunks
        )
        
        # Combine chunks in order
        if results and 'documents' in results and results['documents']:
            return "\n\n".join([doc for doc in results['documents'][0] if doc])
        
        return f"No content found for standard: {standard_name}"
    
    def list_available_standards(self) -> List[str]:
        """
        List all available standards in the database.
        
        Returns:
            A list of standard names.
        """
        if not self.collection:
            return ["Error: Vector database not properly initialized."]
        
        # This is a simplified approach - in reality you'd need a more robust method
        # to extract unique standard names from metadata
        try:
            results = self.collection.get()
            if results and 'metadatas' in results and results['metadatas']:
                standards = set()
                for metadata in results['metadatas']:
                    if 'source' in metadata:
                        standards.add(metadata['source'])
                return list(standards)
        except Exception as e:
            return [f"Error listing standards: {e}"]
        
        return []

In [15]:
class AAOIFIStandardsEnhancementSystem:
    """Main system coordinating the multi-agent process."""
    
    def __init__(self):
        # Initialize vector database manager
        self.db_manager = VectorDBManager()
        
        # Initialize agents
        self.review_agent = ReviewAgent()
        self.enhancement_agent = EnhancementAgent()
        self.validation_agent = ValidationAgent()
        self.report_agent = FinalReportAgent()
        
        # Track processing results
        self.results = {}
    
    def list_available_standards(self) -> List[str]:
        """List all available standards in the system."""
        return self.db_manager.list_available_standards()
    
    def process_standard(self, standard_name: str) -> Dict[str, Any]:
        """
        Process a single standard through the complete pipeline.
        
        Args:
            standard_name: The name of the standard to process.
            
        Returns:
            A dictionary containing the final results.
        """
        print(f"Processing standard: {standard_name}")
        
        # Step 1: Get standard content from the vector database
        print("Retrieving standard content...")
        content = self.db_manager.get_standard_content(standard_name)
        standard = StandardDocument(name=standard_name, content=content)
        
        # Step 2: Review and extract key elements
        print("Reviewing standard and extracting key elements...")
        review_result = self.review_agent.execute(standard)
        self.results["review_result"] = review_result
        
        # Step 3: Propose enhancements
        print("Proposing enhancements...")
        enhancement_result = self.enhancement_agent.execute(review_result)
        self.results["enhancement_proposals"] = enhancement_result["enhancement_proposals"]
        
        # Step 4: Validate proposed changes
        print("Validating proposed changes...")
        validation_result = self.validation_agent.execute(enhancement_result, review_result)
        self.results["validation_result"] = validation_result["validation_result"]
        
        # Step 5: Generate final report
        print("Generating final report...")
        all_results = {
            "standard_name": standard_name,
            "review_result": review_result["review_result"],
            "enhancement_proposals": enhancement_result["enhancement_proposals"],
            "validation_result": validation_result["validation_result"]
        }
        
        final_report = self.report_agent.execute(all_results)
        self.results["final_report"] = final_report["final_report"]
        self.results["standard_name"] = standard_name
        
        print(f"Processing completed for standard: {standard_name}")
        return self.results
    
    def save_results(self, output_dir: str = config.OUTPUT_DIR):
        """Save all results to output files."""
        os.makedirs(output_dir, exist_ok=True)
        
        if not self.results:
            print("No results to save.")
            return
        
        standard_name = self.results.get("standard_name", "unknown_standard")
        filename = os.path.join(output_dir, f"{standard_name}_results.json")
        
        with open(filename, 'w') as f:
            json.dump(self.results, f, indent=2)
        
        print(f"Results saved to {filename}")
        
        # Also save the final report separately
        if "final_report" in self.results:
            report_filename = os.path.join(output_dir, f"{standard_name}_final_report.md")
            with open(report_filename, 'w') as f:
                f.write(self.results["final_report"])
            
            print(f"Final report saved to {report_filename}")

In [16]:
# First, check if we have PDF files in the pdf_eng directory
pdf_files = [f for f in os.listdir(config.PDF_FOLDER) if f.lower().endswith('.pdf')]
print(f"Found {len(pdf_files)} PDF files in the {config.PDF_FOLDER} directory:")
for pdf in pdf_files:
    print(f"- {pdf}")

Found 49 PDF files in the pdf_eng directory:
- AAOIFI-19th-SB-Conf.-Arabic-Agenda-V9.pdf
- AAOIFI-20th-SB-Conf.-Arabic-Agenda-V5LR.pdf
- AAOIFI-Conceptual-Framework-for-Financial-Reporting-Revised-2020-Final-clean.pdf
- AAOIFI-SB-Conf.-17-Booklet-A-Rev-5-LR.pdf
- AAOIFI-SB-Conf.-17-Final-Recommendations.pdf
- AAOIFI-WB-Conf.-13th-Ara-AgendaFinal-2.pdf
- Conceptual-Framework-for-Financial-Reporting-by-Islamic-Financial-Institutions.pdf
- FAS-1-General-Presentation-and-Disclosure-in-the-Financial-Statements-of-Islamic-Banks-and-Financial-Institutions.pdf
- FAS-1-General-Presentation-and-Disclosures-in-the-Financial-Statements-_-v7-clean-17-October-2022.pdf
- FAS-28-Murabaha-and-Other-Deferred-Payment-Sales-Formatted-2021-clean-1.pdf
- FAS-30-Impairment-Credit-Losses-and-Onerous-Commitments-Formatted-2021-clean-13-Nov-22-1.pdf
- FAS-31-Investment-Agency-Al-Wakala-Bi-Al-Istithmar-Final-format-2021-updated-clean-1.pdf
- FAS-32-Ijarah-Formatted-2021-clean-April-2023-1.pdf
- FAS-33-Investment

In [17]:
# # Process the PDFs to create the vector database
# # Only run this cell when you want to process PDFs
# process_pdfs()

In [18]:
# Initialize the system
system = AAOIFIStandardsEnhancementSystem()

# List available standards
print("Available standards:")
standards = system.list_available_standards()
for i, standard in enumerate(standards):
    print(f"{i+1}. {standard}")

C:\Users\PC\AppData\Local\Temp\ipykernel_19276\2179003765.py:7: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  self.embeddings = OpenAIEmbeddings()
C:\Users\PC\AppData\Local\Temp\ipykernel_19276\3989169443.py:13: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  self.llm = ChatOpenAI(model_name=model_name, temperature=0.2)


Available standards:
1. FAS-1-General-Presentation-and-Disclosure-in-the-Financial-Statements-of-Islamic-Banks-and-Financial-Institutions
2. FAS-44_-Determining-Control-of-Assets-and-Business-Final
3. FAS-28-Murabaha-and-Other-Deferred-Payment-Sales-Formatted-2021-clean-1
4. FAS-1-General-Presentation-and-Disclosures-in-the-Financial-Statements-_-v7-clean-17-October-2022
5. AAOIFI-SB-Conf.-17-Final-Recommendations
6. AAOIFI-19th-SB-Conf.-Arabic-Agenda-V9
7. FAS-42-Presentation-and-Disclosures-in-the-Financial-Statements-of-Takaful-Institutions-final-clean
8. FAS-40-Financial-Reporting-for-Islamic-Windows
9. FAS-32-Ijarah-Formatted-2021-clean-April-2023-1
10. FAS-43-Accounting-for-Takaful-Recognition-and-Measurement_Final_clean
11. FAS-30-Impairment-Credit-Losses-and-Onerous-Commitments-Formatted-2021-clean-13-Nov-22-1
12. FAS-39-Financial-reporting-for-Zakah-clean-june-2022
13. AAOIFI-WB-Conf.-13th-Ara-AgendaFinal-2
14. FAS-41-Interim-Financial-Reporting-v16-final-for-issuance_clean
15

In [19]:
# Option 1: Fix the dimension mismatch by clearing the database and recreating it

def recreate_vector_database(documents):
    """Create a new vector database from document chunks after removing any existing data."""
    import os
    import shutil
    from langchain.embeddings import OpenAIEmbeddings
    import chromadb
    
    # Initialize embeddings provider
    embeddings = OpenAIEmbeddings(model=config.EMBEDDING_MODEL)
    
    # First, check if the DB directory exists and delete it
    if os.path.exists(config.DB_DIRECTORY):
        print(f"Removing existing database directory: {config.DB_DIRECTORY}")
        shutil.rmtree(config.DB_DIRECTORY)
    
    # Create a new directory
    os.makedirs(config.DB_DIRECTORY, exist_ok=True)
    
    # Create Chroma client with new empty directory
    client = chromadb.PersistentClient(path=config.DB_DIRECTORY)
    
    # Create new collection
    collection = client.create_collection(name=config.COLLECTION_NAME)
    
    # Process documents in batches to avoid API limits
    batch_size = 100
    for i in range(0, len(documents), batch_size):
        batch = documents[i:i+batch_size]
        
        # Extract content and metadata
        texts = [doc["content"] for doc in batch]
        ids = [f"doc_{i+j}" for j in range(len(batch))]
        metadatas = [doc["metadata"] for doc in batch]
        
        # Generate embeddings 
        embeds = embeddings.embed_documents(texts)
        
        # Add to collection
        collection.add(
            embeddings=embeds,
            documents=texts,
            ids=ids,
            metadatas=metadatas
        )
    
    print(f"Created new vector database with consistent embedding dimensions in '{config.DB_DIRECTORY}'")
    return client

# Option 2: Alternative approach to fix VectorDBManager
class VectorDBManager:
    """Manager for interacting with the vector database."""

    def __init__(self, db_directory: str = config.DB_DIRECTORY, collection_name: str = config.COLLECTION_NAME):
        self.db_directory = db_directory
        self.collection_name = collection_name
        self.embeddings = OpenAIEmbeddings(model=config.EMBEDDING_MODEL)  # Use consistent model
        
        # Initialize client
        self.client = chromadb.PersistentClient(path=db_directory)
        
        # Get collection or recreate it if there's an error
        try:
            self.collection = self.client.get_collection(name=collection_name)
            print(f"Successfully connected to existing collection: {collection_name}")
        except Exception as e:
            print(f"Error accessing collection: {e}")
            print("Creating a new collection...")
            self.collection = self.client.create_collection(name=collection_name)
            
    def get_standard_content(self, standard_name: str) -> str:
        """
        Retrieve the content for a specific standard.
        
        Args:
            standard_name: The name of the standard to retrieve.
            
        Returns:
            The combined content of the standard.
        """
        if not self.collection:
            return "Error: Vector database not properly initialized."
            
        # Query for all chunks belonging to this standard
        # For a new or recreated collection, we may need to handle empty results
        try:
            results = self.collection.query(
                query_texts=[""],  # Empty query to get all documents
                where={"source": standard_name},
                n_results=100  # Adjust based on expected number of chunks
            )
            
            # Combine chunks in order
            if results and 'documents' in results and results['documents'] and results['documents'][0]:
                return "\n\n".join([doc for doc in results['documents'][0] if doc])
            else:
                return f"No content found for standard: {standard_name}"
        except Exception as e:
            return f"Error querying for standard {standard_name}: {str(e)}"

    def list_available_standards(self) -> List[str]:
        """
        List all available standards in the database.
        
        Returns:
            A list of standard names.
        """
        if not self.collection:
            return ["Error: Vector database not properly initialized."]
            
        # Handle potentially empty collection
        try:
            results = self.collection.get()
            if results and 'metadatas' in results and results['metadatas']:
                standards = set()
                for metadata in results['metadatas']:
                    if 'source' in metadata:
                        standards.add(metadata['source'])
                return list(standards)
            else:
                return ["No standards found in the database. Please process PDFs first."]
        except Exception as e:
            return [f"Error listing standards: {str(e)}"]

# Modified process_pdfs function to use the recreate_vector_database function
def process_pdfs():
    """Main function to process PDFs and create vector database."""
    PDF_FOLDER = config.PDF_FOLDER

    if not os.path.exists(PDF_FOLDER):
        print(f"Error: The folder '{PDF_FOLDER}' does not exist.")
        return

    all_documents = []
    pdf_files = [f for f in os.listdir(PDF_FOLDER) if f.lower().endswith('.pdf')]

    if not pdf_files:
        print(f"No PDF files found in '{PDF_FOLDER}'.")
        return

    print(f"Found {len(pdf_files)} PDF files.")

    for pdf_file in pdf_files:
        pdf_path = os.path.join(PDF_FOLDER, pdf_file)
        standard_name = os.path.splitext(pdf_file)[0]

        print(f"Processing {pdf_file}...")

        # Extract text from PDF
        raw_text = extract_text_from_pdf(pdf_path)

        # Clean the text
        cleaned_text = clean_text(raw_text)

        # Split into chunks
        chunks = split_text_into_chunks(cleaned_text, standard_name)
        all_documents.extend(chunks)

        print(f"Extracted {len(chunks)} chunks from {pdf_file}")

    print(f"Total chunks extracted: {len(all_documents)}")

    # Create vector database, recreating it from scratch to ensure consistent dimensions
    print("Creating vector database...")
    client = recreate_vector_database(all_documents)

    print(f"Vector database created successfully in '{config.DB_DIRECTORY}'")
    return client

In [20]:
# # Process PDFs to create a fresh vector database
# process_pdfs()

# # Initialize the system with the updated VectorDBManager
# system = AAOIFIStandardsEnhancementSystem()

# # Verify the available standards
# standards = system.list_available_standards()
# print("Available standards:")
# for i, standard in enumerate(standards):
#     print(f"{i+1}. {standard}")

In [21]:
# Modified approach to handle locked files

def safely_recreate_vector_database(documents):
    """Create a new vector database from document chunks with proper handling of locked files."""
    import os
    import time
    import random
    import string
    from langchain.embeddings import OpenAIEmbeddings
    import chromadb
    
    # Initialize embeddings provider
    embeddings = OpenAIEmbeddings(model=config.EMBEDDING_MODEL)
    
    # Instead of deleting the directory, create a new directory with a unique name
    # This avoids file permission issues with locked files
    timestamp = time.strftime("%Y%m%d_%H%M%S")
    random_str = ''.join(random.choices(string.ascii_lowercase + string.digits, k=6))
    new_db_directory = f"{config.DB_DIRECTORY}_{timestamp}_{random_str}"
    
    print(f"Creating new database directory: {new_db_directory}")
    os.makedirs(new_db_directory, exist_ok=True)
    
    # Create Chroma client with the new directory
    client = chromadb.PersistentClient(path=new_db_directory)
    
    # Create new collection
    collection = client.create_collection(name=config.COLLECTION_NAME)
    
    # Process documents in batches to avoid API limits
    batch_size = 100
    for i in range(0, len(documents), batch_size):
        batch = documents[i:i+batch_size]
        
        # Extract content and metadata
        texts = [doc["content"] for doc in batch]
        ids = [f"doc_{i+j}" for j in range(len(batch))]
        metadatas = [doc["metadata"] for doc in batch]
        
        try:
            # Generate embeddings 
            embeds = embeddings.embed_documents(texts)
            
            # Add to collection
            collection.add(
                embeddings=embeds,
                documents=texts,
                ids=ids,
                metadatas=metadatas
            )
            print(f"Processed batch {i//batch_size + 1} ({len(batch)} documents)")
        except Exception as e:
            print(f"Error processing batch starting at index {i}: {str(e)}")
            continue
    
    print(f"Created new vector database with consistent embedding dimensions in '{new_db_directory}'")
    
    # Update the config to point to the new directory
    config.DB_DIRECTORY = new_db_directory
    print(f"Updated configuration to use new database directory: {config.DB_DIRECTORY}")
    
    return client

# Modified process_pdfs function to use the safely_recreate_vector_database function
def process_pdfs_safe():
    """Main function to process PDFs and create vector database safely."""
    PDF_FOLDER = config.PDF_FOLDER

    if not os.path.exists(PDF_FOLDER):
        print(f"Error: The folder '{PDF_FOLDER}' does not exist.")
        return

    all_documents = []
    pdf_files = [f for f in os.listdir(PDF_FOLDER) if f.lower().endswith('.pdf')]

    if not pdf_files:
        print(f"No PDF files found in '{PDF_FOLDER}'.")
        return

    print(f"Found {len(pdf_files)} PDF files.")

    for pdf_file in pdf_files:
        pdf_path = os.path.join(PDF_FOLDER, pdf_file)
        standard_name = os.path.splitext(pdf_file)[0]

        print(f"Processing {pdf_file}...")

        # Extract text from PDF
        raw_text = extract_text_from_pdf(pdf_path)

        # Clean the text
        cleaned_text = clean_text(raw_text)

        # Split into chunks
        chunks = split_text_into_chunks(cleaned_text, standard_name)
        all_documents.extend(chunks)

        print(f"Extracted {len(chunks)} chunks from {pdf_file}")

    print(f"Total chunks extracted: {len(all_documents)}")

    # Create vector database using the safe approach
    print("Creating vector database safely...")
    client = safely_recreate_vector_database(all_documents)

    print(f"Vector database created successfully in '{config.DB_DIRECTORY}'")
    return client

# Modified VectorDBManager class to work with the new directory and handle potential issues
class VectorDBManager:
    """Manager for interacting with the vector database."""

    def __init__(self, db_directory: str = None, collection_name: str = config.COLLECTION_NAME):
        # Use the directory specified in config (which may have been updated)
        self.db_directory = db_directory if db_directory else config.DB_DIRECTORY
        self.collection_name = collection_name
        self.embeddings = OpenAIEmbeddings(model=config.EMBEDDING_MODEL)  # Use consistent model
        
        print(f"Initializing VectorDBManager with database directory: {self.db_directory}")
        
        # Try to initialize client with better error handling
        try:
            self.client = chromadb.PersistentClient(path=self.db_directory)
            print(f"Successfully connected to database at: {self.db_directory}")
        except Exception as e:
            print(f"Error connecting to database: {str(e)}")
            self.client = None
            self.collection = None
            return
            
        # Try to get the collection with better error handling
        try:
            self.collection = self.client.get_collection(name=collection_name)
            print(f"Successfully connected to collection: {collection_name}")
        except Exception as e:
            print(f"Error accessing collection: {str(e)}")
            try:
                print(f"Attempting to create a new collection: {collection_name}")
                self.collection = self.client.create_collection(name=collection_name)
                print(f"New collection created successfully")
            except Exception as e2:
                print(f"Error creating collection: {str(e2)}")
                self.collection = None
                
    def get_standard_content(self, standard_name: str) -> str:
        """Retrieve the content for a specific standard."""
        if not self.collection:
            return "Error: Vector database not properly initialized."
            
        try:
            results = self.collection.query(
                query_texts=[""],  # Empty query to get all documents
                where={"source": standard_name},
                n_results=100  # Adjust based on expected number of chunks
            )
            
            if results and 'documents' in results and results['documents'] and results['documents'][0]:
                return "\n\n".join([doc for doc in results['documents'][0] if doc])
            else:
                return f"No content found for standard: {standard_name}"
        except Exception as e:
            return f"Error querying for standard {standard_name}: {str(e)}"

    def list_available_standards(self) -> List[str]:
        """List all available standards in the database."""
        if not self.collection:
            return ["Error: Vector database not properly initialized."]
            
        try:
            results = self.collection.get()
            if results and 'metadatas' in results and results['metadatas']:
                standards = set()
                for metadata in results['metadatas']:
                    if 'source' in metadata:
                        standards.add(metadata['source'])
                return list(standards)
            else:
                return ["No standards found in the database. Please process PDFs first."]
        except Exception as e:
            return [f"Error listing standards: {str(e)}"]

# Implementation to test the new approach
def test_safe_approach():
    """Test the safe approach to creating and using the vector database."""
    
    # Process PDFs using the safe approach
    print("Processing PDFs with safe approach...")
    client = process_pdfs_safe()
    
    # Initialize system with the new database
    print("\nInitializing AAOIFIStandardsEnhancementSystem...")
    system = AAOIFIStandardsEnhancementSystem()
    
    # Check available standards
    print("\nListing available standards...")
    standards = system.list_available_standards()
    for i, standard in enumerate(standards):
        print(f"{i+1}. {standard}")
    
    # If any standards are available, process the first one
    if standards and standards[0] != "No standards found in the database. Please process PDFs first.":
        standard_name = standards[0]
        print(f"\nProcessing standard: {standard_name}")
        results = system.process_standard(standard_name)
        system.save_results()
        
        # Display the final report
        if "final_report" in results:
            print("\n===== FINAL REPORT =====\n")
            print(results["final_report"])
    else:
        print("\nNo standards available to process.")
        
    return "Test completed."

In [22]:
# Execute the safe approach test
test_safe_approach()

Processing PDFs with safe approach...
Found 49 PDF files.
Processing AAOIFI-19th-SB-Conf.-Arabic-Agenda-V9.pdf...
Extracted 3 chunks from AAOIFI-19th-SB-Conf.-Arabic-Agenda-V9.pdf
Processing AAOIFI-20th-SB-Conf.-Arabic-Agenda-V5LR.pdf...
Extracted 3 chunks from AAOIFI-20th-SB-Conf.-Arabic-Agenda-V5LR.pdf
Processing AAOIFI-Conceptual-Framework-for-Financial-Reporting-Revised-2020-Final-clean.pdf...
Extracted 178 chunks from AAOIFI-Conceptual-Framework-for-Financial-Reporting-Revised-2020-Final-clean.pdf
Processing AAOIFI-SB-Conf.-17-Booklet-A-Rev-5-LR.pdf...
Extracted 9 chunks from AAOIFI-SB-Conf.-17-Booklet-A-Rev-5-LR.pdf
Processing AAOIFI-SB-Conf.-17-Final-Recommendations.pdf...
Extracted 1 chunks from AAOIFI-SB-Conf.-17-Final-Recommendations.pdf
Processing AAOIFI-WB-Conf.-13th-Ara-AgendaFinal-2.pdf...
Extracted 10 chunks from AAOIFI-WB-Conf.-13th-Ara-AgendaFinal-2.pdf
Processing Conceptual-Framework-for-Financial-Reporting-by-Islamic-Financial-Institutions.pdf...
Extracted 240 chunks

C:\Users\PC\AppData\Local\Temp\ipykernel_19276\3989169443.py:58: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=self.llm, prompt=prompt)
C:\Users\PC\AppData\Local\Temp\ipykernel_19276\3989169443.py:59: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = chain.run({})


APIConnectionError: Connection error.

In [23]:
# Complete workflow for processing standards

# Step 1: Initialize your system
system = AAOIFIStandardsEnhancementSystem()

# Step 2: List all available standards to know what you can process
print("Available standards:")
standards = system.list_available_standards()
for i, standard in enumerate(standards):
    print(f"{i+1}. {standard}")

# Step 3: Process a specific standard by index or name
# Option A: Process by index (e.g., process the first standard)
if standards and standards[0] != "No standards found in the database. Please process PDFs first.":
    # Process the first standard in the list
    standard_name = standards[0]
    print(f"\nProcessing standard: {standard_name}")
    results = system.process_standard(standard_name)
    system.save_results()
    
    # Display the final report
    if "final_report" in results:
        print("\n===== FINAL REPORT =====\n")
        print(results["final_report"])

# Option B: Process by specific name (replace with actual standard name)
specific_standard = "Your_Standard_Name_Here"  # Replace with a name from your standards list
if specific_standard in standards:
    print(f"\nProcessing specific standard: {specific_standard}")
    results = system.process_standard(specific_standard)
    system.save_results()
    
    # Display the final report
    if "final_report" in results:
        print("\n===== FINAL REPORT =====\n")
        print(results["final_report"])
else:
    print(f"Standard '{specific_standard}' not found in available standards.")

# Step 4: Process all standards in batch (optional)
def process_all_standards():
    """Process all available standards one by one."""
    print("\nProcessing all standards in batch mode...")
    
    all_results = {}
    for standard in standards:
        if standard != "No standards found in the database. Please process PDFs first.":
            print(f"\nProcessing standard: {standard}")
            try:
                results = system.process_standard(standard)
                system.save_results()
                all_results[standard] = results
                print(f"✓ Successfully processed: {standard}")
            except Exception as e:
                print(f"✗ Error processing standard {standard}: {str(e)}")
    
    print("\nBatch processing completed!")
    print(f"Successfully processed {len(all_results)} standards.")
    return all_results

# Uncomment the following line to process all standards
# all_results = process_all_standards()

# Step 5: View results for all processed standards
def list_processed_results():
    """List all processed results in the results directory."""
    import os
    
    results_dir = config.OUTPUT_DIR
    if not os.path.exists(results_dir):
        print(f"Results directory '{results_dir}' does not exist.")
        return
        
    result_files = [f for f in os.listdir(results_dir) if f.endswith("_results.json") or f.endswith("_final_report.md")]
    
    if not result_files:
        print(f"No result files found in '{results_dir}'.")
        return
        
    print(f"\nFound {len(result_files)} result files:")
    for i, result_file in enumerate(result_files):
        print(f"{i+1}. {result_file}")
        
    return result_files

# Uncomment to list all processed results
# result_files = list_processed_results()

# Step 6: Read a specific result file
def read_report(report_name):
    """Read and display a specific report."""
    import os
    import json
    
    results_dir = config.OUTPUT_DIR
    report_path = os.path.join(results_dir, report_name)
    
    if not os.path.exists(report_path):
        print(f"Report file '{report_path}' does not exist.")
        return None
        
    print(f"Reading report: {report_name}")
    
    if report_name.endswith("_final_report.md"):
        # Read markdown report
        with open(report_path, 'r') as f:
            report_content = f.read()
        print("\n===== REPORT CONTENT =====\n")
        print(report_content)
        return report_content
    elif report_name.endswith("_results.json"):
        # Read JSON results
        with open(report_path, 'r') as f:
            results = json.load(f)
        print("\n===== RESULTS SUMMARY =====\n")
        print(f"Standard Name: {results.get('standard_name', 'Unknown')}")
        print(f"Contains Final Report: {'Yes' if 'final_report' in results else 'No'}")
        print(f"Contains Review Result: {'Yes' if 'review_result' in results else 'No'}")
        print(f"Contains Enhancement Proposals: {'Yes' if 'enhancement_proposals' in results else 'No'}")
        print(f"Contains Validation Result: {'Yes' if 'validation_result' in results else 'No'}")
        return results
    else:
        print(f"Unsupported file format: {report_name}")
        return None

# Example: Read a specific report (uncomment and replace with actual report name)
# read_report("standard_name_final_report.md")

Initializing VectorDBManager with database directory: aaoifi_vector_db_20250510_171304_gqfbl3
Successfully connected to database at: aaoifi_vector_db_20250510_171304_gqfbl3
Successfully connected to collection: aaoifi_standards
Available standards:
1. FAS-1-General-Presentation-and-Disclosure-in-the-Financial-Statements-of-Islamic-Banks-and-Financial-Institutions
2. FAS-28-Murabaha-and-Other-Deferred-Payment-Sales-Formatted-2021-clean-1
3. FAS-1-General-Presentation-and-Disclosures-in-the-Financial-Statements-_-v7-clean-17-October-2022
4. AAOIFI-SB-Conf.-17-Final-Recommendations
5. AAOIFI-19th-SB-Conf.-Arabic-Agenda-V9
6. FAS-42-Presentation-and-Disclosures-in-the-Financial-Statements-of-Takaful-Institutions-final-clean
7. FAS-40-Financial-Reporting-for-Islamic-Windows
8. FAS-32-Ijarah-Formatted-2021-clean-April-2023-1
9. FAS-30-Impairment-Credit-Losses-and-Onerous-Commitments-Formatted-2021-clean-13-Nov-22-1
10. FAS-39-Financial-reporting-for-Zakah-clean-june-2022
11. AAOIFI-WB-Conf.-

APIConnectionError: Connection error.

In [24]:
print("Available standards:")
standards = system.list_available_standards()
for i, standard in enumerate(standards):
    print(f"{i+1}. {standard}")

Available standards:
1. FAS-1-General-Presentation-and-Disclosure-in-the-Financial-Statements-of-Islamic-Banks-and-Financial-Institutions
2. FAS-28-Murabaha-and-Other-Deferred-Payment-Sales-Formatted-2021-clean-1
3. FAS-1-General-Presentation-and-Disclosures-in-the-Financial-Statements-_-v7-clean-17-October-2022
4. AAOIFI-SB-Conf.-17-Final-Recommendations
5. AAOIFI-19th-SB-Conf.-Arabic-Agenda-V9
6. FAS-42-Presentation-and-Disclosures-in-the-Financial-Statements-of-Takaful-Institutions-final-clean
7. FAS-40-Financial-Reporting-for-Islamic-Windows
8. FAS-32-Ijarah-Formatted-2021-clean-April-2023-1
9. FAS-30-Impairment-Credit-Losses-and-Onerous-Commitments-Formatted-2021-clean-13-Nov-22-1
10. FAS-39-Financial-reporting-for-Zakah-clean-june-2022
11. AAOIFI-WB-Conf.-13th-Ara-AgendaFinal-2
12. FAS-41-Interim-Financial-Reporting-v16-final-for-issuance_clean
13. FAS-38-Waad-Khiyar-and-Tahawwut-Final-15-December-2020-clean
14. Conceptual-Framework-for-Financial-Reporting-by-Islamic-Financial-In

In [ ]:
class StandardDocument:
    """Class to represent an AAOIFI standard document."""
    def __init__(self, name: str, content: str):
        self.name = name
        self.content = content

class BaseAgent:
    """Base class for all agents in the system."""
    def __init__(self, name: str, description: str, model_name: str = Config.GPT4_MODEL):
        self.name = name
        self.description = description
        self.model_name = model_name
        # Ensure API key is set before initializing ChatOpenAI
        if not os.environ.get("OPENAI_API_KEY"):
            raise ValueError("OPENAI_API_KEY not set in environment.")
        self.llm = ChatOpenAI(model_name=model_name, temperature=0.2)


    def execute(self, input_data: Any) -> Any:
        """Execute the agent's task."""
        raise NotImplementedError("Subclasses must implement this method")

class ReviewAgent(BaseAgent):
    """Agent responsible for reviewing standards and extracting key elements."""

    def __init__(self):
        super().__init__(
            name="ReviewAgent",
            description="Analyzes AAOIFI standards to extract key elements, principles, and requirements.",
            model_name=Config.GPT4_MODEL
        )

        self.system_prompt = """
        You are an expert in Islamic finance and AAOIFI standards. Your task is to carefully review
        the provided standard document and extract the following key elements.
        Use clear Markdown headings for each section exactly as listed below (e.g., ## Core principles and objectives).

        ## Core principles and objectives
        [Your extraction here]

        ## Key definitions and terminology
        [Your extraction here]
        
        # ... (and so on for other sections) ...

        Be thorough but concise.
        """
    def execute(self, standard: StandardDocument) -> Dict[str, Any]:
        """
        Analyze a standard document and extract its key elements.

        Args:
            standard: The standard document to analyze.

        Returns:
            A dictionary containing the extracted key elements.
        """
        prompt = ChatPromptTemplate.from_messages([
            SystemMessage(content=self.system_prompt),
            HumanMessage(content=f"Standard Name: {standard.name}\n\nContent:\n{standard.content}")
        ])

        chain = LLMChain(llm=self.llm, prompt=prompt)
        # The chain.run({}) is problematic if the prompt itself needs variables.
        # Here, standard.name and standard.content are already formatted into the HumanMessage.
        # If the prompt template had input_variables, they would be passed to run().
        result_text = chain.run({}) # Assuming the prompt is self-contained after formatting.

        # Try to parse the result into structured data (simple approach)
        parsed_result = {
            "standard_name": standard.name,
            "review_result": result_text, # This is the full text from LLM
            "core_principles": self._extract_section(result_text, "Core principles and objectives"),
            "key_definitions": self._extract_section(result_text, "Key definitions and terminology"),
            "main_requirements": self._extract_section(result_text, "Main requirements and procedures"),
            "compliance_criteria": self._extract_section(result_text, "Compliance criteria and guidelines"), # "guidelines" was missing
            "implementation_considerations": self._extract_section(result_text, "Practical implementation considerations") # "Practical" was missing
        }
        return parsed_result


    def _extract_section(self, text: str, section_name: str) -> str:
        """Helper method to extract specific sections from the review result using regex."""
        # Regex to find a heading (e.g., "1. Section Name" or "Section Name:") and capture text until the next heading or end of string
        # This assumes headings are followed by a newline.
        # It also makes section_name matching case-insensitive.
        pattern = re.compile(
            r"(?i)(?:^\s*\d*\.?\s*|\n\s*\d*\.?\s*)" + re.escape(section_name) + r"\s*[:\n](.*?)(?=\n\s*\d*\.?\s*\w+.*?\s*[:\n]|\Z)",
            re.DOTALL | re.IGNORECASE
        )
        match = pattern.search(text)
        if match:
            return match.group(1).strip()
        return f"Section '{section_name}' not found or parsing error."


class EnhancementAgent(BaseAgent):
    """Agent tasked with proposing modifications or enhancements to standards."""

    def __init__(self):
        super().__init__(
            name="EnhancementAgent",
            description="Proposes AI-driven modifications and enhancements to AAOIFI standards.",
            model_name=Config.GPT4_MODEL
        )

        self.system_prompt = """
        You are an AI expert specializing in Islamic finance and AAOIFI standards enhancement.
        Your task is to propose thoughtful modifications and enhancements to the standard based
        on the review provided.

        Consider the following aspects in your proposals:

        1. Clarity improvements: Suggest clearer language or better organization where appropriate
        2. Modern context adaptations: Propose updates to address contemporary financial practices
        3. Technological integration: Recommend ways to incorporate digital technologies and fintech
        4. Cross-reference enhancements: Suggest improved links to related standards or principles
        5. Practical implementation: Provide more actionable guidance for practitioners

        For each suggestion, provide:
        - The specific section or clause being enhanced (if applicable, otherwise general proposal)
        - The current text or concept (if applicable, very brief summary)
        - Your proposed modification or addition
        - A brief justification explaining the benefit of your enhancement

        Structure your response clearly, perhaps using bullet points for each proposal.
        Ensure all suggestions maintain strict compliance with Shariah principles.
        """

    def execute(self, review_result: Dict[str, Any]) -> Dict[str, Any]:
        """
        Propose enhancements to a standard based on the review.

        Args:
            review_result: The result from the ReviewAgent (the full dictionary).

        Returns:
            A dictionary containing proposed enhancements.
        """
        # We need the text of the review, not the whole dict, for the prompt
        review_text_summary = review_result.get("review_result", "No review summary available.")

        prompt = ChatPromptTemplate.from_messages([
            SystemMessage(content=self.system_prompt),
            HumanMessage(content=f"Standard Name: {review_result['standard_name']}\n\nSummary of Standard Review:\n{review_text_summary}")
        ])

        chain = LLMChain(llm=self.llm, prompt=prompt)
        result_text = chain.run({})

        return {
            "standard_name": review_result["standard_name"],
            "enhancement_proposals": result_text # This is the LLM's textual output
        }

class ValidationAgent(BaseAgent):
    """Agent for validating and approving proposed changes."""

    def __init__(self):
        super().__init__(
            name="ValidationAgent",
            description="Validates proposed changes based on Shariah compliance and practicality.",
            model_name=Config.GPT4_MODEL
        )

        self.system_prompt = """
        You are a senior Shariah scholar and AAOIFI standards expert. Your role is to validate
        proposed enhancements to ensure they maintain compliance with Islamic principles and
        practical applicability.

        For each proposed enhancement, evaluate:

        1. Shariah Compliance: Does the proposal align with Islamic principles and AAOIFI's mission?
        2. Technical Accuracy: Is the proposed language precise and technically sound?
        3. Practical Applicability: Can the enhancement be practically implemented by Islamic financial institutions?
        4. Consistency: Does it maintain consistency with other standards and established practices?
        5. Value Addition: Does it meaningfully improve the standard?

        For each proposal, provide:
        - Your assessment (e.g., Approved, Rejected, Needs Modification)
        - A detailed justification for your decision, referencing the evaluation criteria above.
        - Suggested refinements if "Needs Modification".

        Be thorough in your analysis and maintain the highest standards of Islamic finance integrity.
        Structure your response clearly, addressing each proposal from the input.
        """

    def execute(self, enhancement_result: Dict[str, Any], original_review: Dict[str, Any]) -> Dict[str, Any]:
        """
        Validate proposed enhancements based on Shariah compliance and practicality.

        Args:
            enhancement_result: The result from the EnhancementAgent (dictionary).
            original_review: The original review from the ReviewAgent (dictionary).

        Returns:
            A dictionary containing validation results.
        """
        review_text_summary = original_review.get("review_result", "No review summary available.")
        enhancement_proposals_text = enhancement_result.get("enhancement_proposals", "No enhancement proposals available.")

        prompt = ChatPromptTemplate.from_messages([
            SystemMessage(content=self.system_prompt),
            HumanMessage(content=f"""
            Standard Name: {enhancement_result['standard_name']}

            Original Standard Review Summary:
            {review_text_summary}

            Proposed Enhancements to Validate:
            {enhancement_proposals_text}
            """)
        ])

        chain = LLMChain(llm=self.llm, prompt=prompt)
        result_text = chain.run({})

        return {
            "standard_name": enhancement_result["standard_name"],
            "validation_result": result_text # This is the LLM's textual output
        }

class FinalReportAgent(BaseAgent):
    """Agent for generating a comprehensive final report."""

    def __init__(self):
        super().__init__(
            name="FinalReportAgent",
            description="Compiles all findings and recommendations into a comprehensive report.",
            model_name=Config.GPT4_MODEL # GPT-4 for high-quality report generation
        )

        self.system_prompt = """
        You are a professional report writer specializing in Islamic finance standards. Your task is to
        compile all the findings, enhancements, and validations into a comprehensive, well-structured report
        in Markdown format.

        Your report should include the following sections:

        1.  **Executive Summary**: Brief overview of the standard reviewed, key findings, summary of proposed changes, and validation outcomes.
        2.  **Standard Overview**: Summary of the original standard's purpose and core components (based on the review).
        3.  **Key Findings from Review**: Major elements and considerations identified during the initial review.
        4.  **Proposed Enhancements**: Clear presentation of all proposed modifications.
        5.  **Validation Results**: Summary of the validation process and outcomes for each proposal.
        6.  **Consolidated Recommendations**: A final list of approved or modified enhancements.
        7.  **Implementation Considerations**: Practical next steps for adopting approved changes.
        8.  **Conclusion**: Final thoughts on the impact of the proposed enhancements.

        Write in a professional, clear, and objective style appropriate for AAOIFI stakeholders and Islamic finance professionals.
        Use Markdown for formatting (headings, lists, bolding).
        """

    def execute(self, all_results: Dict[str, Any]) -> Dict[str, Any]:
        """
        Generate a comprehensive final report.

        Args:
            all_results: Combined results from all previous agents.
                         Expected keys: 'standard_name', 'review_text',
                                        'enhancements_text', 'validation_text'.

        Returns:
            A dictionary containing the final report text.
        """
        prompt = ChatPromptTemplate.from_messages([
            SystemMessage(content=self.system_prompt),
            HumanMessage(content=f"""
            Standard Name: {all_results['standard_name']}

            Full Text of Standard Review:
            {all_results['review_text']}

            Full Text of Proposed Enhancements:
            {all_results['enhancements_text']}

            Full Text of Validation of Enhancements:
            {all_results['validation_text']}
            """)
        ])

        chain = LLMChain(llm=self.llm, prompt=prompt)
        report_text = chain.run({})

        return {
            "standard_name": all_results["standard_name"],
            "final_report": report_text # This is the LLM's textual output (Markdown)
        }


class VectorDBManager:
    """Manager for interacting with the vector database."""

    def __init__(self, db_directory: str = None, collection_name: str = config.COLLECTION_NAME):
        self.db_directory = db_directory if db_directory else config.DB_DIRECTORY
        self.collection_name = collection_name
        self.embeddings = OpenAIEmbeddings(model=config.EMBEDDING_MODEL)

        print(f"Initializing VectorDBManager with database directory: {self.db_directory}")

        try:
            if not os.path.exists(self.db_directory):
                print(f"Database directory {self.db_directory} does not exist. Creating it.")
                os.makedirs(self.db_directory, exist_ok=True)
            self.client = chromadb.PersistentClient(path=self.db_directory)
            print(f"Successfully created PersistentClient for database at: {self.db_directory}")
        except Exception as e:
            print(f"Fatal error creating PersistentClient for {self.db_directory}: {str(e)}")
            self.client = None
            self.collection = None
            raise  # Reraise critical error

        if self.client:
            try:
                # Try to get the collection; if it fails, try to create it.
                self.collection = self.client.get_or_create_collection(name=self.collection_name)
                print(f"Successfully accessed/created collection: {self.collection_name}")
            except Exception as e:
                print(f"Error accessing/creating collection '{self.collection_name}': {str(e)}")
                self.collection = None
        else:
            self.collection = None


    def get_standard_content(self, standard_name: str) -> str:
        """
        Retrieve the content for a specific standard.

        Args:
            standard_name: The name of the standard to retrieve.

        Returns:
            The combined content of the standard, or an error message.
        """
        if not self.collection:
            return "Error: Vector database collection not properly initialized."

        try:
            # Query for all chunks belonging to this standard
            # A large n_results is needed if a standard has many chunks
            # Chroma's default n_results is 10.
            # To get ALL documents matching a filter, it's better to use collection.get()
            results = self.collection.get(
                where={"source": standard_name},
                include=["documents"] # Only fetch documents
            )

            if results and results['documents']:
                # Sort by chunk_id if available in metadata, though .get() doesn't easily allow sorting
                # For now, just join them. If order is critical, store chunk_id and sort post-retrieval.
                # The current split_text_into_chunks adds metadata, but .get() returns raw docs.
                # Let's assume for now that the order from .get() is sufficient or re-query with sort if needed.
                # The current query in `create_vector_database` uses IDs like `doc_{i+j}` which are sequential.
                # If we rely on implicit order of `get`, it should be mostly fine.
                
                # The metadata is not directly available here to sort by chunk_id without another call or more complex query
                # For simplicity, joining in received order.
                standard_content = "\n\n".join(doc for doc in results['documents'] if doc)
                if not standard_content:
                    return f"No document content found for standard: {standard_name}, though metadatas might exist."
                return standard_content
            else:
                return f"No content found for standard: {standard_name}"
        except Exception as e:
            return f"Error querying for standard {standard_name}: {str(e)}"

    def list_available_standards(self) -> List[str]:
        """
        List all available standards in the database.

        Returns:
            A list of standard names, or an error message list.
        """
        if not self.collection:
            return ["Error: Vector database collection not properly initialized."]

        try:
            results = self.collection.get(include=["metadatas"]) # Fetch only metadatas
            if results and results['metadatas']:
                standards = set()
                for metadata in results['metadatas']:
                    if metadata and 'source' in metadata: # Check if metadata is not None
                        standards.add(metadata['source'])
                if not standards:
                    return ["No standards found in the database. Metadatas might be empty or lack 'source'."]
                return sorted(list(standards))
            else:
                return ["No standards found in the database. Please process PDFs first."]
        except Exception as e:
            return [f"Error listing standards: {str(e)}"]


class AAOIFIStandardsEnhancementSystem:
    """Main system coordinating the multi-agent process."""

    def __init__(self):
        # Initialize vector database manager
        # It will use config.DB_DIRECTORY which might be updated by process_pdfs_safe
        self.db_manager = VectorDBManager()

        # Initialize agents
        self.review_agent = ReviewAgent()
        self.enhancement_agent = EnhancementAgent()
        self.validation_agent = ValidationAgent()
        self.report_agent = FinalReportAgent()

        # Track processing results for a single standard run
        self.current_processing_results = {}

    def list_available_standards(self) -> List[str]:
        """List all available standards in the system."""
        return self.db_manager.list_available_standards()

    def process_standard(self, standard_name: str) -> Dict[str, Any]:
        """
        Process a single standard through the complete pipeline.

        Args:
            standard_name: The name of the standard to process.

        Returns:
            A dictionary containing the final results for this standard.
        """
        print(f"\nProcessing standard: {standard_name}")
        self.current_processing_results = {"standard_name": standard_name}

        # Step 1: Get standard content from the vector database
        print("  Retrieving standard content...")
        content = self.db_manager.get_standard_content(standard_name)
        if content.startswith("Error:") or content.startswith("No content found"):
            print(f"  Error retrieving content for {standard_name}: {content}")
            self.current_processing_results["error"] = content
            return self.current_processing_results
        
        standard_doc = StandardDocument(name=standard_name, content=content)
        self.current_processing_results["original_content_excerpt"] = content[:500] + "..." if len(content) > 500 else content

        # Step 2: Review and extract key elements
        print("  Reviewing standard and extracting key elements...")
        review_output = self.review_agent.execute(standard_doc)
        self.current_processing_results["review_output"] = review_output # Store the whole dict

        # Step 3: Propose enhancements
        print("  Proposing enhancements...")
        enhancement_output = self.enhancement_agent.execute(review_output) # Pass the dict
        self.current_processing_results["enhancement_output"] = enhancement_output

        # Step 4: Validate proposed changes
        print("  Validating proposed changes...")
        # Validation agent needs enhancement_output (dict) and review_output (dict)
        validation_output = self.validation_agent.execute(enhancement_output, review_output)
        self.current_processing_results["validation_output"] = validation_output

        # Step 5: Generate final report
        print("  Generating final report...")
        # Prepare data for report agent
        report_input_data = {
            "standard_name": standard_name,
            "review_text": review_output.get("review_result", "N/A"),
            "enhancements_text": enhancement_output.get("enhancement_proposals", "N/A"),
            "validation_text": validation_output.get("validation_result", "N/A")
        }
        final_report_output = self.report_agent.execute(report_input_data)
        self.current_processing_results["final_report_output"] = final_report_output

        print(f"Processing completed for standard: {standard_name}")
        return self.current_processing_results

    def save_results(self, results_to_save: Dict[str, Any], output_dir: str = config.OUTPUT_DIR):
        """Save all results to output files."""
        os.makedirs(output_dir, exist_ok=True)

        if not results_to_save:
            print("No results to save.")
            return

        standard_name = results_to_save.get("standard_name", "unknown_standard")
        # Sanitize standard_name for filename
        safe_standard_name = re.sub(r'[^\w\-_\. ]', '_', standard_name)
        timestamp = time.strftime("%Y%m%d_%H%M%S")
        
        filename_json = os.path.join(output_dir, f"{safe_standard_name}_full_results_{timestamp}.json")
        with open(filename_json, 'w', encoding='utf-8') as f:
            json.dump(results_to_save, f, indent=2, ensure_ascii=False)
        print(f"Full results saved to {filename_json}")

        # Also save the final report separately as Markdown
        final_report_text = results_to_save.get("final_report_output", {}).get("final_report")
        if final_report_text:
            filename_md = os.path.join(output_dir, f"{safe_standard_name}_final_report_{timestamp}.md")
            with open(filename_md, 'w', encoding='utf-8') as f:
                f.write(final_report_text)
            print(f"Final report saved to {filename_md}")


# --- Functions for recreating vector database safely ---
def safely_recreate_vector_database(documents):
    """Create a new vector database from document chunks with proper handling of locked files."""
    embeddings = OpenAIEmbeddings(model=config.EMBEDDING_MODEL)

    # Create a new directory with a unique name to avoid conflicts
    timestamp = time.strftime("%Y%m%d_%H%M%S")
    random_str = ''.join(random.choices(string.ascii_lowercase + string.digits, k=6))
    # Base directory for all DB versions
    base_db_parent_dir = "aaoifi_vector_db_versions"
    os.makedirs(base_db_parent_dir, exist_ok=True)
    new_db_directory = os.path.join(base_db_parent_dir, f"db_{timestamp}_{random_str}")

    print(f"Creating new database in directory: {new_db_directory}")
    os.makedirs(new_db_directory, exist_ok=True)

    client = chromadb.PersistentClient(path=new_db_directory)
    # Always create a new collection in a new DB path
    collection = client.create_collection(name=config.COLLECTION_NAME)
    
    batch_size = 100 # As used in create_vector_database
    num_batches = (len(documents) + batch_size - 1) // batch_size

    for i in range(0, len(documents), batch_size):
        batch_documents = documents[i:i+batch_size]
        current_batch_num = (i // batch_size) + 1
        print(f"Processing batch {current_batch_num}/{num_batches} for new vector database...")

        texts = [doc["content"] for doc in batch_documents]
        # Use more robust IDs tied to document source and chunk
        ids = [f"{doc['metadata']['source']}_chunk_{doc['metadata']['chunk_id']}" for doc in batch_documents]
        # Check for ID uniqueness within the batch (essential for Chroma)
        if len(ids) != len(set(ids)):
            print(f"Warning: Duplicate IDs generated in batch {current_batch_num}. This may cause issues.")
            # Simple fix: append index within batch to ensure uniqueness for this add op
            ids = [f"{id_}_{j}" for j, id_ in enumerate(ids)]

        metadatas = [doc["metadata"] for doc in batch_documents]

        if not texts:
            print(f"Skipping empty batch {current_batch_num}.")
            continue
        try:
            embeds = embeddings.embed_documents(texts)
            collection.add(embeddings=embeds, documents=texts, ids=ids, metadatas=metadatas)
        except Exception as e:
            print(f"Error processing batch {current_batch_num} starting at index {i}: {str(e)}")
            continue

    print(f"Created new vector database with consistent embedding dimensions in '{new_db_directory}'")
    
    # Update the global config to point to the new directory for the current session
    config.DB_DIRECTORY = new_db_directory
    print(f"Updated global config.DB_DIRECTORY to: {config.DB_DIRECTORY}")
    
    return client

def process_pdfs_safe():
    """Main function to process PDFs and create vector database safely."""
    PDF_FOLDER = config.PDF_FOLDER
    if not os.path.exists(PDF_FOLDER):
        print(f"Error: The PDF folder '{PDF_FOLDER}' does not exist.")
        return None
    if not os.listdir(PDF_FOLDER):
        print(f"The PDF folder '{PDF_FOLDER}' is empty. Please add PDF files to process.")
        return None

    all_documents = []
    pdf_files = [f for f in os.listdir(PDF_FOLDER) if f.lower().endswith('.pdf')]
    if not pdf_files:
        print(f"No PDF files found in '{PDF_FOLDER}'.")
        return None

    print(f"Found {len(pdf_files)} PDF files.")
    for pdf_file in pdf_files:
        pdf_path = os.path.join(PDF_FOLDER, pdf_file)
        standard_name = os.path.splitext(pdf_file)[0]
        print(f"Processing {pdf_file}...")
        raw_text = extract_text_from_pdf(pdf_path)
        cleaned_text = clean_text(raw_text)
        chunks = split_text_into_chunks(cleaned_text, standard_name)
        all_documents.extend(chunks)
        print(f"Extracted {len(chunks)} chunks from {pdf_file}")

    if not all_documents:
        print("No documents were extracted from PDFs. Cannot create vector database.")
        return None
        
    print(f"Total chunks extracted: {len(all_documents)}")
    print("Creating/Recreating vector database safely...")
    client = safely_recreate_vector_database(all_documents) # This updates config.DB_DIRECTORY
    print(f"Vector database operations completed. Current DB directory: '{config.DB_DIRECTORY}'")
    return client

# --- START OF NEW DEMONSTRATION CODE ---
"""
AAOIFI Standards Enhancement System Demo

This script demonstrates how to use the multi-agent architecture to:
1.  Select an AAOIFI standard
2.  Review and extract key elements using ReviewAgent
3.  Propose AI-driven modifications/enhancements using EnhancementAgent
4.  Validate proposed changes based on Shariah principles using ValidationAgent
5.  Generate a comprehensive report using FinalReportAgent
6.  Optionally, use additional agents for visualization and feedback simulation.
"""

class DemoRunner:
    """Class to run a complete demonstration of the AAOIFI Standards Enhancement System."""
    
    def __init__(self, system: AAOIFIStandardsEnhancementSystem):
        self.system = system
        self.selected_standard_name: Optional[str] = None
        self.current_demo_results: Dict[str, Any] = {} # Stores results for the current demo run
        
    def _get_excerpt(self, text: Optional[str], max_length: int = 300) -> str:
        """Get an excerpt of text with maximum length."""
        if not text:
            return "N/A"
        text = str(text) # Ensure it's a string
        if len(text) <= max_length:
            return text
        return text[:max_length] + "..."

    def list_and_select_standard(self) -> bool:
        """List all available standards and allow selection."""
        print("\n" + "="*50)
        print("AAOIFI STANDARDS ENHANCEMENT SYSTEM DEMONSTRATION")
        print("="*50)
        
        print("\nAvailable standards:")
        standards = self.system.list_available_standards()
        
        if not standards or any(s.startswith("Error:") or s.startswith("No standards found") for s in standards):
            print("No standards available or error listing standards.")
            print("Please ensure PDFs are processed and a vector database exists.")
            if standards: print(f"Details: {standards[0]}")
            return False
            
        for i, standard_name in enumerate(standards):
            print(f"{i+1}. {standard_name}")
        
        while True:
            try:
                choice_str = input(f"\nSelect a standard to process (enter number 1-{len(standards)}): ")
                if not choice_str: continue # Handle empty input
                choice = int(choice_str)
                if 1 <= choice <= len(standards):
                    self.selected_standard_name = standards[choice-1]
                    print(f"\nSelected standard: {self.selected_standard_name}")
                    self.current_demo_results = {"standard_name": self.selected_standard_name} # Reset for new selection
                    return True
                else:
                    print(f"Invalid selection. Please enter a number between 1 and {len(standards)}.")
            except ValueError:
                print("Invalid input. Please enter a number.")
    
    def run_review_phase(self) -> bool:
        """Run the review phase and display results."""
        if not self.selected_standard_name:
            print("Error: No standard selected for review phase.")
            return False

        print("\n" + "-"*50)
        print("PHASE 1: STANDARD REVIEW AND ANALYSIS (ReviewAgent)")
        print("-"*50)
        
        print(f"\nRetrieving content for standard: {self.selected_standard_name}...")
        content = self.system.db_manager.get_standard_content(self.selected_standard_name)
        if content.startswith("Error:") or content.startswith("No content found"):
            print(f"  Error retrieving content: {content}")
            self.current_demo_results["review_error"] = content
            return False
        
        standard_document = StandardDocument(name=self.selected_standard_name, content=content)
        
        print(f"Reviewing standard using ReviewAgent...")
        start_time = time.time()
        review_output = self.system.review_agent.execute(standard_document)
        self.current_demo_results["review_output"] = review_output
        elapsed_time = time.time() - start_time
        print(f"ReviewAgent completed in {elapsed_time:.2f} seconds.")
        
        print("\nREVIEW SUMMARY:")
        print(f"- Standard: {review_output.get('standard_name', 'N/A')}")
        print(f"\nCore Principles (excerpt):\n{self._get_excerpt(review_output.get('core_principles'))}")
        print(f"\nKey Definitions (excerpt):\n{self._get_excerpt(review_output.get('key_definitions'))}")
        # print(f"\nFull Review Text (excerpt):\n{self._get_excerpt(review_output.get('review_result'), 500)}")
        return True
    
    def run_enhancement_phase(self) -> bool:
        """Run the enhancement phase and display results."""
        if "review_output" not in self.current_demo_results:
            print("Error: Review phase must complete successfully before enhancement.")
            return False

        print("\n" + "-"*50)
        print("PHASE 2: AI-DRIVEN ENHANCEMENT PROPOSALS (EnhancementAgent)")
        print("-"*50)
        
        review_output = self.current_demo_results["review_output"]
        print(f"\nGenerating enhancement proposals for: {self.selected_standard_name} using EnhancementAgent...")
        
        start_time = time.time()
        enhancement_output = self.system.enhancement_agent.execute(review_output) # Pass full review dict
        self.current_demo_results["enhancement_output"] = enhancement_output
        elapsed_time = time.time() - start_time
        print(f"EnhancementAgent completed in {elapsed_time:.2f} seconds.")
        
        print("\nENHANCEMENT PROPOSALS (excerpt):")
        print(self._get_excerpt(enhancement_output.get("enhancement_proposals"), 500))
        return True

    def run_validation_phase(self) -> bool:
        """Run the validation phase and display results."""
        if "enhancement_output" not in self.current_demo_results or \
           "review_output" not in self.current_demo_results:
            print("Error: Review and Enhancement phases must complete successfully before validation.")
            return False

        print("\n" + "-"*50)
        print("PHASE 3: SHARIAH COMPLIANCE VALIDATION (ValidationAgent)")
        print("-"*50)

        review_output = self.current_demo_results["review_output"]
        enhancement_output = self.current_demo_results["enhancement_output"]
        
        print(f"\nValidating proposed enhancements for: {self.selected_standard_name} using ValidationAgent...")
        start_time = time.time()
        validation_output = self.system.validation_agent.execute(enhancement_output, review_output)
        self.current_demo_results["validation_output"] = validation_output
        elapsed_time = time.time() - start_time
        print(f"ValidationAgent completed in {elapsed_time:.2f} seconds.")
        
        print("\nVALIDATION RESULTS (excerpt):")
        print(self._get_excerpt(validation_output.get("validation_result"), 500))
        return True
    
    def run_report_generation_phase(self) -> bool:
        """Generate the final comprehensive report."""
        if "validation_output" not in self.current_demo_results: # Implies previous phases also ran
            print("Error: All previous phases (Review, Enhance, Validate) must complete before report generation.")
            return False

        print("\n" + "-"*50)
        print("PHASE 4: COMPREHENSIVE REPORT GENERATION (FinalReportAgent)")
        print("-"*50)
        
        print(f"\nGenerating comprehensive report for: {self.selected_standard_name} using FinalReportAgent...")
        
        report_input_data = {
            "standard_name": self.selected_standard_name,
            "review_text": self.current_demo_results.get("review_output", {}).get("review_result", "N/A"),
            "enhancements_text": self.current_demo_results.get("enhancement_output", {}).get("enhancement_proposals", "N/A"),
            "validation_text": self.current_demo_results.get("validation_output", {}).get("validation_result", "N/A")
        }
        
        start_time = time.time()
        final_report_output = self.system.report_agent.execute(report_input_data)
        self.current_demo_results["final_report_output"] = final_report_output
        elapsed_time = time.time() - start_time
        print(f"FinalReportAgent completed in {elapsed_time:.2f} seconds.")
        
        print("\nFINAL REPORT (excerpt):")
        print(self._get_excerpt(final_report_output.get("final_report"), 600))
        return True

    def save_demo_results(self):
        """Save all results from the demonstration."""
        if not self.current_demo_results or not self.selected_standard_name:
            print("No results to save or standard not selected.")
            return

        print("\n" + "-"*50)
        print("SAVING DEMONSTRATION RESULTS")
        print("-"*50)
        
        # Use the system's save_results method, passing the demo's collected results
        self.system.save_results(self.current_demo_results) # It handles directory creation and naming
    
    def run_complete_demo(self):
        """Run the complete demonstration process."""
        if not self.list_and_select_standard():
            print("Demo terminated: Standard selection failed or was aborted.")
            return
        
        if not self.run_review_phase():
            print("Demo terminated: Review phase failed.")
            return
        
        if not self.run_enhancement_phase():
            print("Demo terminated: Enhancement phase failed.")
            return
        
        if not self.run_validation_phase():
            print("Demo terminated: Validation phase failed.")
            return
        
        if not self.run_report_generation_phase():
            print("Demo terminated: Report generation phase failed.")
            return
        
        self.save_demo_results()
        
        print("\n" + "="*50)
        print(f"DEMONSTRATION COMPLETED SUCCESSFULLY FOR STANDARD: {self.selected_standard_name}")
        print("="*50)
        print(f"All results and the final report have been saved to the '{config.OUTPUT_DIR}' directory.")

# --- Additional Agents for Enhanced Demo ---

class VisualizationAgent(BaseAgent):
    """Agent responsible for generating textual descriptions for visualizations."""
    
    def __init__(self):
        super().__init__(
            name="VisualizationAgent",
            description="Creates textual summaries suitable for generating visual representations of standard changes.",
            model_name=Config.GPT35_MODEL # Faster model for summarization
        )
        
        self.system_prompt = """
        You are a data visualization assistant. Your task is to process the provided summaries of AAOIFI standard reviews,
        enhancement proposals, and validation results, and then generate concise textual descriptions that could be used
        to create visual elements (like tables or charts).

        Focus on:
        1.  A summary table of key proposed changes: Section | Original Concept | Proposed Change | Benefit.
        2.  A Shariah compliance assessment summary: Proposal Area | Compliance Status | Key Shariah Considerations.
        3.  A benefits overview: Key Enhancement | Primary Benefit to Stakeholders.

        Present this information clearly using Markdown tables or structured lists.
        """
    
    def _get_excerpt(self, text: Optional[str], max_length: int = 300) -> str: # Helper from DemoRunner
        if not text: return "N/A"
        text = str(text)
        if len(text) <= max_length: return text
        return text[:max_length] + "..."

    def execute(self, demo_results: Dict[str, Any]) -> Dict[str, Any]:
        standard_name = demo_results.get('standard_name', 'N/A')
        review_text = self._get_excerpt(demo_results.get('review_output', {}).get('review_result', 'N/A'), 1000)
        enhancements_text = self._get_excerpt(demo_results.get('enhancement_output', {}).get('enhancement_proposals', 'N/A'), 1000)
        validation_text = self._get_excerpt(demo_results.get('validation_output', {}).get('validation_result', 'N/A'), 1000)

        prompt = ChatPromptTemplate.from_messages([
            SystemMessage(content=self.system_prompt),
            HumanMessage(content=f"""
            Standard Name: {standard_name}
            
            Summary of Standard Review:
            {review_text}
            
            Summary of Proposed Enhancements:
            {enhancements_text}
            
            Summary of Validation Results:
            {validation_text}
            
            Please generate textual descriptions suitable for visualization based on the above.
            """)
        ])
        
        chain = LLMChain(llm=self.llm, prompt=prompt)
        visualization_text_data = chain.run({})
        
        return {
            "standard_name": standard_name,
            "visualization_text_data": visualization_text_data
        }

class FeedbackAgent(BaseAgent):
    """Agent for collecting and analyzing simulated feedback on proposed changes."""
    
    def __init__(self):
        super().__init__(
            name="FeedbackAgent",
            description="Processes simulated feedback on proposed standard changes and generates insights.",
            model_name=Config.GPT35_MODEL
        )
        
        self.system_prompt = """
        You are an expert in qualitative feedback analysis for financial standards.
        Given a set of simulated feedback entries on proposed changes to an AAOIFI standard,
        your task is to:

        1.  Summarize the overall sentiment (Positive, Negative, Mixed).
        2.  Identify 2-3 key themes or concerns raised by stakeholders.
        3.  Highlight 2-3 constructive suggestions for further improvement, if any.
        4.  Note any areas of strong consensus or significant disagreement.

        Provide a concise analysis.
        """
    
    def execute(self, standard_name: str, simulated_feedback_list: List[Dict[str, Any]]) -> Dict[str, Any]:
        formatted_feedback = "\n\n".join([
            f"Feedback Entry #{i+1}:\nStakeholder Role: {item.get('role', 'N/A')}\nRating: {item.get('rating', 'N/A')}/5\nComment: {item.get('comment', 'N/A')}"
            for i, item in enumerate(simulated_feedback_list)
        ])
        
        prompt = ChatPromptTemplate.from_messages([
            SystemMessage(content=self.system_prompt),
            HumanMessage(content=f"""
            Standard Name: {standard_name}
            
            Simulated Stakeholder Feedback:
            {formatted_feedback}
            
            Please analyze this feedback.
            """)
        ])
        
        chain = LLMChain(llm=self.llm, prompt=prompt)
        analysis_result = chain.run({})
        
        avg_rating = sum(item.get('rating', 0) for item in simulated_feedback_list) / len(simulated_feedback_list) if simulated_feedback_list else 0
        
        return {
            "standard_name": standard_name,
            "feedback_count": len(simulated_feedback_list),
            "average_simulated_rating": f"{avg_rating:.2f}/5.00",
            "feedback_analysis_summary": analysis_result
        }

class EnhancedDemoRunner(DemoRunner):
    """Enhanced demo runner with visualization and feedback capabilities."""
    
    def __init__(self, system: AAOIFIStandardsEnhancementSystem):
        super().__init__(system)
        self.visualization_agent = VisualizationAgent()
        self.feedback_agent = FeedbackAgent()
        
    def run_visualization_phase(self) -> bool:
        """Generate textual descriptions for visualizations based on the results."""
        if not self.current_demo_results.get("final_report_output"): # Check if prior phases are done
            print("Error: Core demo phases must complete before visualization.")
            return False

        print("\n" + "-"*50)
        print("PHASE 5: GENERATING VISUALIZATION DATA (VisualizationAgent)")
        print("-"*50)
        
        print(f"\nGenerating visualization text for: {self.selected_standard_name} using VisualizationAgent...")
        start_time = time.time()
        visualization_output = self.visualization_agent.execute(self.current_demo_results)
        self.current_demo_results["visualization_output"] = visualization_output
        elapsed_time = time.time() - start_time
        print(f"VisualizationAgent completed in {elapsed_time:.2f} seconds.")
        
        print("\nVISUALIZATION TEXT DATA (excerpt):")
        print(self._get_excerpt(visualization_output.get("visualization_text_data"), 500))
        return True
    
    def run_simulated_feedback_phase(self) -> bool:
        """Simulate stakeholder feedback and analyze it."""
        if not self.current_demo_results.get("final_report_output"):
            print("Error: Core demo phases must complete before feedback simulation.")
            return False

        print("\n" + "-"*50)
        print("PHASE 6: SIMULATED FEEDBACK ANALYSIS (FeedbackAgent)")
        print("-"*50)
        
        print(f"\nSimulating stakeholder feedback for: {self.selected_standard_name}...")
        simulated_feedback = [
            {"role": "Shariah Scholar", "rating": 4, "comment": "The proposals are largely compliant, but clarification is needed on the application of 'gharar yaseer' in proposed digital contracts."},
            {"role": "Banker", "rating": 5, "comment": "These changes will significantly improve operational efficiency and reduce ambiguity in sukuk issuance."},
            {"role": "Regulator", "rating": 3, "comment": "While the intent is good, the proposed technological integrations might pose supervisory challenges. We need more detailed risk management guidelines."},
            {"role": "Academic", "rating": 4, "comment": "A solid step forward. I suggest including references to contemporary research on fintech in Islamic finance for a more robust theoretical underpinning."}
        ]
        print(f"Generated {len(simulated_feedback)} simulated feedback entries.")

        print(f"Analyzing simulated feedback using FeedbackAgent...")
        start_time = time.time()
        feedback_analysis_output = self.feedback_agent.execute(self.selected_standard_name, simulated_feedback)
        self.current_demo_results["feedback_analysis_output"] = feedback_analysis_output
        elapsed_time = time.time() - start_time
        print(f"FeedbackAgent completed in {elapsed_time:.2f} seconds.")

        print("\nSIMULATED FEEDBACK ANALYSIS (excerpt):")
        print(self._get_excerpt(feedback_analysis_output.get("feedback_analysis_summary"), 500))
        return True

    def run_complete_enhanced_demo(self):
        """Run the complete enhanced demonstration process."""
        if not self.list_and_select_standard():
            print("Enhanced Demo terminated: Standard selection failed.")
            return
        
        if not self.run_review_phase():
            print("Enhanced Demo terminated: Review phase failed.")
            return
        
        if not self.run_enhancement_phase():
            print("Enhanced Demo terminated: Enhancement phase failed.")
            return
        
        if not self.run_validation_phase():
            print("Enhanced Demo terminated: Validation phase failed.")
            return
        
        if not self.run_report_generation_phase():
            print("Enhanced Demo terminated: Report generation phase failed.")
            return
        
        # Enhanced Steps
        if not self.run_visualization_phase():
            print("Enhanced Demo warning: Visualization phase failed, but core demo completed.")
        
        if not self.run_simulated_feedback_phase():
            print("Enhanced Demo warning: Feedback simulation phase failed, but core demo completed.")
            
        self.save_demo_results()
        
        print("\n" + "="*50)
        print(f"ENHANCED DEMONSTRATION COMPLETED SUCCESSFULLY FOR STANDARD: {self.selected_standard_name}")
        print("="*50)
        print(f"All results, including enhanced analyses, have been saved to the '{config.OUTPUT_DIR}' directory.")


# Main execution block for the demo
def main_demo():
    """Main function to run the demonstration."""
    print("Initializing AAOIFI Standards Enhancement System for Demo...")
    
    # Step 1: Ensure Vector DB is populated (or try to populate it)
    # Try to initialize system first. If it fails badly (e.g. DB dir issue), then try to process PDFs.
    try:
        system_check = AAOIFIStandardsEnhancementSystem()
        standards_check = system_check.list_available_standards()
        if not standards_check or any(s.startswith("Error:") or s.startswith("No standards found") for s in standards_check):
            print("\nNo standards readily available or error accessing existing DB.")
            run_pdf_processing = input("Attempt to process PDFs in 'pdf_eng' folder to create/recreate database? (yes/no): ").strip().lower()
            if run_pdf_processing == 'yes':
                print("Processing PDFs using safe recreate method...")
                process_pdfs_safe() # This updates config.DB_DIRECTORY
                print("Re-initializing system with the new/updated database...")
                # System will be initialized below with the potentially new config.DB_DIRECTORY
            else:
                print("PDF processing skipped. Demo may not function if no standards are available.")
        else:
            print(f"Found existing standards: {standards_check[:3]}...") # Print first few
    except Exception as e:
        print(f"Initial system/DB check failed: {e}")
        run_pdf_processing = input("Attempt to process PDFs in 'pdf_eng' folder to create/recreate database? (yes/no): ").strip().lower()
        if run_pdf_processing == 'yes':
            print("Processing PDFs using safe recreate method...")
            process_pdfs_safe() # This updates config.DB_DIRECTORY
            print("Re-initializing system with the new/updated database...")
        else:
            print("PDF processing skipped. Demo cannot continue without a functional database.")
            return

    # Initialize the main system for the demo (picks up current config.DB_DIRECTORY)
    try:
        system = AAOIFIStandardsEnhancementSystem()
    except Exception as e:
        print(f"Fatal error initializing AAOIFIStandardsEnhancementSystem: {e}")
        print("Please check your database setup and OpenAI API key.")
        return

    # Choose which demo to run
    print("\nSelect Demo Type:")
    print("1. Basic Demo (Review, Enhance, Validate, Report)")
    print("2. Enhanced Demo (includes Visualization and Feedback Analysis)")
    
    while True:
        try:
            choice_str = input("Enter your choice (1 or 2, or 'exit'): ").strip()
            if choice_str.lower() == 'exit':
                print("Exiting demo.")
                break
            if not choice_str: continue

            choice = int(choice_str)
            if choice == 1:
                demo = DemoRunner(system)
                demo.run_complete_demo()
                break
            elif choice == 2:
                demo = EnhancedDemoRunner(system)
                demo.run_complete_enhanced_demo()
                break
            else:
                print("Invalid selection. Please enter 1 or 2.")
        except ValueError:
            print("Invalid input. Please enter a number (1 or 2) or 'exit'.")
        except Exception as e:
            print(f"An unexpected error occurred during demo execution: {e}")
            # traceback.print_exc() # For debugging
            break

if __name__ == "__main__":
    # Ensure OPENAI_API_KEY is set
    if not os.environ.get("OPENAI_API_KEY") or "sk-proj-" not in os.environ.get("OPENAI_API_KEY"): # Basic check
        print("Error: OPENAI_API_KEY is not set or appears invalid in the script.")
        print("Please set it near the top of the script (line 24 approx).")
        # You might want to exit here if the key is critical for all operations
        # exit() # Uncomment if you want to force exit

    # Check if pdf_eng folder exists and has PDFs, offer to run process_pdfs_safe if empty
    pdf_folder = Config.PDF_FOLDER
    if not os.path.exists(pdf_folder):
        print(f"PDF folder '{pdf_folder}' does not exist. Please create it and add AAOIFI standard PDFs.")
    elif not [f for f in os.listdir(pdf_folder) if f.lower().endswith('.pdf')]:
        print(f"PDF folder '{pdf_folder}' is empty. Please add AAOIFI standard PDFs to process.")
    
    # Run the main demonstration logic
    main_demo()

In [26]:

class StandardDocument:
    """Class to represent an AAOIFI standard document."""
    def __init__(self, name: str, content: str):
        self.name = name
        self.content = content

class BaseAgent:
    """Base class for all agents in the system."""
    def __init__(self, name: str, description: str, model_name: str = Config.GPT4_MODEL):
        self.name = name
        self.description = description
        self.model_name = model_name
        # Ensure API key is set before initializing ChatOpenAI
        if not os.environ.get("OPENAI_API_KEY"):
            raise ValueError("OPENAI_API_KEY not set in environment.")
        self.llm = ChatOpenAI(model_name=model_name, temperature=0.2)


    def execute(self, input_data: Any) -> Any:
        """Execute the agent's task."""
        raise NotImplementedError("Subclasses must implement this method")

class ReviewAgent(BaseAgent):
    """Agent responsible for reviewing standards and extracting key elements."""

    def __init__(self):
        super().__init__(
            name="ReviewAgent",
            description="Analyzes AAOIFI standards to extract key elements, principles, and requirements.",
            model_name=Config.GPT4_MODEL
        )

        self.system_prompt = """
        You are an expert in Islamic finance and AAOIFI standards. Your task is to carefully review
        the provided standard document and extract the following key elements:

        1. Core principles and objectives of the standard
        2. Key definitions and terminology
        3. Main requirements and procedures
        4. Compliance criteria and guidelines
        5. Practical implementation considerations

        Organize your analysis in a structured format with these categories. Be thorough but concise
        in your extraction of the essential components. Respond with the extracted information directly,
        using markdown for headings for each category.
        """

    def execute(self, standard: StandardDocument) -> Dict[str, Any]:
        """
        Analyze a standard document and extract its key elements.

        Args:
            standard: The standard document to analyze.

        Returns:
            A dictionary containing the extracted key elements.
        """
        prompt = ChatPromptTemplate.from_messages([
            SystemMessage(content=self.system_prompt),
            HumanMessage(content=f"Standard Name: {standard.name}\n\nContent:\n{standard.content}")
        ])

        chain = LLMChain(llm=self.llm, prompt=prompt)
        # The chain.run({}) is problematic if the prompt itself needs variables.
        # Here, standard.name and standard.content are already formatted into the HumanMessage.
        # If the prompt template had input_variables, they would be passed to run().
        result_text = chain.run({}) # Assuming the prompt is self-contained after formatting.

        # Try to parse the result into structured data (simple approach)
        parsed_result = {
            "standard_name": standard.name,
            "review_result": result_text, # This is the full text from LLM
            "core_principles": self._extract_section(result_text, "Core principles and objectives"),
            "key_definitions": self._extract_section(result_text, "Key definitions and terminology"),
            "main_requirements": self._extract_section(result_text, "Main requirements and procedures"),
            "compliance_criteria": self._extract_section(result_text, "Compliance criteria and guidelines"), # "guidelines" was missing
            "implementation_considerations": self._extract_section(result_text, "Practical implementation considerations") # "Practical" was missing
        }
        return parsed_result


    def _extract_section(self, text: str, section_name: str) -> str:
        """Helper method to extract specific sections from the review result using regex."""
        # Regex to find a heading (e.g., "1. Section Name" or "Section Name:") and capture text until the next heading or end of string
        # This assumes headings are followed by a newline.
        # It also makes section_name matching case-insensitive.
        pattern = re.compile(
            r"(?i)(?:^\s*\d*\.?\s*|\n\s*\d*\.?\s*)" + re.escape(section_name) + r"\s*[:\n](.*?)(?=\n\s*\d*\.?\s*\w+.*?\s*[:\n]|\Z)",
            re.DOTALL | re.IGNORECASE
        )
        match = pattern.search(text)
        if match:
            return match.group(1).strip()
        return f"Section '{section_name}' not found or parsing error."


class EnhancementAgent(BaseAgent):
    """Agent tasked with proposing modifications or enhancements to standards."""

    def __init__(self):
        super().__init__(
            name="EnhancementAgent",
            description="Proposes AI-driven modifications and enhancements to AAOIFI standards.",
            model_name=Config.GPT4_MODEL
        )

        self.system_prompt = """
        You are an AI expert specializing in Islamic finance and AAOIFI standards enhancement.
        Your task is to propose thoughtful modifications and enhancements to the standard based
        on the review provided.

        Consider the following aspects in your proposals:

        1. Clarity improvements: Suggest clearer language or better organization where appropriate
        2. Modern context adaptations: Propose updates to address contemporary financial practices
        3. Technological integration: Recommend ways to incorporate digital technologies and fintech
        4. Cross-reference enhancements: Suggest improved links to related standards or principles
        5. Practical implementation: Provide more actionable guidance for practitioners

        For each suggestion, provide:
        - The specific section or clause being enhanced (if applicable, otherwise general proposal)
        - The current text or concept (if applicable, very brief summary)
        - Your proposed modification or addition
        - A brief justification explaining the benefit of your enhancement

        Structure your response clearly, perhaps using bullet points for each proposal.
        Ensure all suggestions maintain strict compliance with Shariah principles.
        """

    def execute(self, review_result: Dict[str, Any]) -> Dict[str, Any]:
        """
        Propose enhancements to a standard based on the review.

        Args:
            review_result: The result from the ReviewAgent (the full dictionary).

        Returns:
            A dictionary containing proposed enhancements.
        """
        # We need the text of the review, not the whole dict, for the prompt
        review_text_summary = review_result.get("review_result", "No review summary available.")

        prompt = ChatPromptTemplate.from_messages([
            SystemMessage(content=self.system_prompt),
            HumanMessage(content=f"Standard Name: {review_result['standard_name']}\n\nSummary of Standard Review:\n{review_text_summary}")
        ])

        chain = LLMChain(llm=self.llm, prompt=prompt)
        result_text = chain.run({})

        return {
            "standard_name": review_result["standard_name"],
            "enhancement_proposals": result_text # This is the LLM's textual output
        }

class ValidationAgent(BaseAgent):
    """Agent for validating and approving proposed changes."""

    def __init__(self):
        super().__init__(
            name="ValidationAgent",
            description="Validates proposed changes based on Shariah compliance and practicality.",
            model_name=Config.GPT4_MODEL
        )

        self.system_prompt = """
        You are a senior Shariah scholar and AAOIFI standards expert. Your role is to validate
        proposed enhancements to ensure they maintain compliance with Islamic principles and
        practical applicability.

        For each proposed enhancement, evaluate:

        1. Shariah Compliance: Does the proposal align with Islamic principles and AAOIFI's mission?
        2. Technical Accuracy: Is the proposed language precise and technically sound?
        3. Practical Applicability: Can the enhancement be practically implemented by Islamic financial institutions?
        4. Consistency: Does it maintain consistency with other standards and established practices?
        5. Value Addition: Does it meaningfully improve the standard?

        For each proposal, provide:
        - Your assessment (e.g., Approved, Rejected, Needs Modification)
        - A detailed justification for your decision, referencing the evaluation criteria above.
        - Suggested refinements if "Needs Modification".

        Be thorough in your analysis and maintain the highest standards of Islamic finance integrity.
        Structure your response clearly, addressing each proposal from the input.
        """

    def execute(self, enhancement_result: Dict[str, Any], original_review: Dict[str, Any]) -> Dict[str, Any]:
        """
        Validate proposed enhancements based on Shariah compliance and practicality.

        Args:
            enhancement_result: The result from the EnhancementAgent (dictionary).
            original_review: The original review from the ReviewAgent (dictionary).

        Returns:
            A dictionary containing validation results.
        """
        review_text_summary = original_review.get("review_result", "No review summary available.")
        enhancement_proposals_text = enhancement_result.get("enhancement_proposals", "No enhancement proposals available.")

        prompt = ChatPromptTemplate.from_messages([
            SystemMessage(content=self.system_prompt),
            HumanMessage(content=f"""
            Standard Name: {enhancement_result['standard_name']}

            Original Standard Review Summary:
            {review_text_summary}

            Proposed Enhancements to Validate:
            {enhancement_proposals_text}
            """)
        ])

        chain = LLMChain(llm=self.llm, prompt=prompt)
        result_text = chain.run({})

        return {
            "standard_name": enhancement_result["standard_name"],
            "validation_result": result_text # This is the LLM's textual output
        }

class FinalReportAgent(BaseAgent):
    """Agent for generating a comprehensive final report."""

    def __init__(self):
        super().__init__(
            name="FinalReportAgent",
            description="Compiles all findings and recommendations into a comprehensive report.",
            model_name=Config.GPT4_MODEL # GPT-4 for high-quality report generation
        )

        self.system_prompt = """
        You are a professional report writer specializing in Islamic finance standards. Your task is to
        compile all the findings, enhancements, and validations into a comprehensive, well-structured report
        in Markdown format.

        Your report should include the following sections:

        1.  **Executive Summary**: Brief overview of the standard reviewed, key findings, summary of proposed changes, and validation outcomes.
        2.  **Standard Overview**: Summary of the original standard's purpose and core components (based on the review).
        3.  **Key Findings from Review**: Major elements and considerations identified during the initial review.
        4.  **Proposed Enhancements**: Clear presentation of all proposed modifications.
        5.  **Validation Results**: Summary of the validation process and outcomes for each proposal.
        6.  **Consolidated Recommendations**: A final list of approved or modified enhancements.
        7.  **Implementation Considerations**: Practical next steps for adopting approved changes.
        8.  **Conclusion**: Final thoughts on the impact of the proposed enhancements.

        Write in a professional, clear, and objective style appropriate for AAOIFI stakeholders and Islamic finance professionals.
        Use Markdown for formatting (headings, lists, bolding).
        """

    def execute(self, all_results: Dict[str, Any]) -> Dict[str, Any]:
        """
        Generate a comprehensive final report.

        Args:
            all_results: Combined results from all previous agents.
                         Expected keys: 'standard_name', 'review_text',
                                        'enhancements_text', 'validation_text'.

        Returns:
            A dictionary containing the final report text.
        """
        prompt = ChatPromptTemplate.from_messages([
            SystemMessage(content=self.system_prompt),
            HumanMessage(content=f"""
            Standard Name: {all_results['standard_name']}

            Full Text of Standard Review:
            {all_results['review_text']}

            Full Text of Proposed Enhancements:
            {all_results['enhancements_text']}

            Full Text of Validation of Enhancements:
            {all_results['validation_text']}
            """)
        ])

        chain = LLMChain(llm=self.llm, prompt=prompt)
        report_text = chain.run({})

        return {
            "standard_name": all_results["standard_name"],
            "final_report": report_text # This is the LLM's textual output (Markdown)
        }


class VectorDBManager:
    """Manager for interacting with the vector database."""

    def __init__(self, db_directory: str = None, collection_name: str = config.COLLECTION_NAME):
        self.db_directory = db_directory if db_directory else config.DB_DIRECTORY
        self.collection_name = collection_name
        self.embeddings = OpenAIEmbeddings(model=config.EMBEDDING_MODEL)

        print(f"Initializing VectorDBManager with database directory: {self.db_directory}")

        try:
            if not os.path.exists(self.db_directory):
                print(f"Database directory {self.db_directory} does not exist. Creating it.")
                os.makedirs(self.db_directory, exist_ok=True)
            self.client = chromadb.PersistentClient(path=self.db_directory)
            print(f"Successfully created PersistentClient for database at: {self.db_directory}")
        except Exception as e:
            print(f"Fatal error creating PersistentClient for {self.db_directory}: {str(e)}")
            self.client = None
            self.collection = None
            raise  # Reraise critical error

        if self.client:
            try:
                # Try to get the collection; if it fails, try to create it.
                self.collection = self.client.get_or_create_collection(name=self.collection_name)
                print(f"Successfully accessed/created collection: {self.collection_name}")
            except Exception as e:
                print(f"Error accessing/creating collection '{self.collection_name}': {str(e)}")
                self.collection = None
        else:
            self.collection = None


    def get_standard_content(self, standard_name: str) -> str:
        """
        Retrieve the content for a specific standard.

        Args:
            standard_name: The name of the standard to retrieve.

        Returns:
            The combined content of the standard, or an error message.
        """
        if not self.collection:
            return "Error: Vector database collection not properly initialized."

        try:
            # Query for all chunks belonging to this standard
            # A large n_results is needed if a standard has many chunks
            # Chroma's default n_results is 10.
            # To get ALL documents matching a filter, it's better to use collection.get()
            results = self.collection.get(
                where={"source": standard_name},
                include=["documents"] # Only fetch documents
            )

            if results and results['documents']:
                # Sort by chunk_id if available in metadata, though .get() doesn't easily allow sorting
                # For now, just join them. If order is critical, store chunk_id and sort post-retrieval.
                # The current split_text_into_chunks adds metadata, but .get() returns raw docs.
                # Let's assume for now that the order from .get() is sufficient or re-query with sort if needed.
                # The current query in `create_vector_database` uses IDs like `doc_{i+j}` which are sequential.
                # If we rely on implicit order of `get`, it should be mostly fine.
                
                # The metadata is not directly available here to sort by chunk_id without another call or more complex query
                # For simplicity, joining in received order.
                standard_content = "\n\n".join(doc for doc in results['documents'] if doc)
                if not standard_content:
                    return f"No document content found for standard: {standard_name}, though metadatas might exist."
                return standard_content
            else:
                return f"No content found for standard: {standard_name}"
        except Exception as e:
            return f"Error querying for standard {standard_name}: {str(e)}"

    def list_available_standards(self) -> List[str]:
        """
        List all available standards in the database.

        Returns:
            A list of standard names, or an error message list.
        """
        if not self.collection:
            return ["Error: Vector database collection not properly initialized."]

        try:
            results = self.collection.get(include=["metadatas"]) # Fetch only metadatas
            if results and results['metadatas']:
                standards = set()
                for metadata in results['metadatas']:
                    if metadata and 'source' in metadata: # Check if metadata is not None
                        standards.add(metadata['source'])
                if not standards:
                    return ["No standards found in the database. Metadatas might be empty or lack 'source'."]
                return sorted(list(standards))
            else:
                return ["No standards found in the database. Please process PDFs first."]
        except Exception as e:
            return [f"Error listing standards: {str(e)}"]


class AAOIFIStandardsEnhancementSystem:
    """Main system coordinating the multi-agent process."""

    def __init__(self):
        # Initialize vector database manager
        # It will use config.DB_DIRECTORY which might be updated by process_pdfs_safe
        self.db_manager = VectorDBManager()

        # Initialize agents
        self.review_agent = ReviewAgent()
        self.enhancement_agent = EnhancementAgent()
        self.validation_agent = ValidationAgent()
        self.report_agent = FinalReportAgent()

        # Track processing results for a single standard run
        self.current_processing_results = {}

    def list_available_standards(self) -> List[str]:
        """List all available standards in the system."""
        return self.db_manager.list_available_standards()

    def process_standard(self, standard_name: str) -> Dict[str, Any]:
        """
        Process a single standard through the complete pipeline.

        Args:
            standard_name: The name of the standard to process.

        Returns:
            A dictionary containing the final results for this standard.
        """
        print(f"\nProcessing standard: {standard_name}")
        self.current_processing_results = {"standard_name": standard_name}

        # Step 1: Get standard content from the vector database
        print("  Retrieving standard content...")
        content = self.db_manager.get_standard_content(standard_name)
        if content.startswith("Error:") or content.startswith("No content found"):
            print(f"  Error retrieving content for {standard_name}: {content}")
            self.current_processing_results["error"] = content
            return self.current_processing_results
        
        standard_doc = StandardDocument(name=standard_name, content=content)
        self.current_processing_results["original_content_excerpt"] = content[:500] + "..." if len(content) > 500 else content

        # Step 2: Review and extract key elements
        print("  Reviewing standard and extracting key elements...")
        review_output = self.review_agent.execute(standard_doc)
        self.current_processing_results["review_output"] = review_output # Store the whole dict

        # Step 3: Propose enhancements
        print("  Proposing enhancements...")
        enhancement_output = self.enhancement_agent.execute(review_output) # Pass the dict
        self.current_processing_results["enhancement_output"] = enhancement_output

        # Step 4: Validate proposed changes
        print("  Validating proposed changes...")
        # Validation agent needs enhancement_output (dict) and review_output (dict)
        validation_output = self.validation_agent.execute(enhancement_output, review_output)
        self.current_processing_results["validation_output"] = validation_output

        # Step 5: Generate final report
        print("  Generating final report...")
        # Prepare data for report agent
        report_input_data = {
            "standard_name": standard_name,
            "review_text": review_output.get("review_result", "N/A"),
            "enhancements_text": enhancement_output.get("enhancement_proposals", "N/A"),
            "validation_text": validation_output.get("validation_result", "N/A")
        }
        final_report_output = self.report_agent.execute(report_input_data)
        self.current_processing_results["final_report_output"] = final_report_output

        print(f"Processing completed for standard: {standard_name}")
        return self.current_processing_results

    def save_results(self, results_to_save: Dict[str, Any], output_dir: str = config.OUTPUT_DIR):
        """Save all results to output files."""
        os.makedirs(output_dir, exist_ok=True)

        if not results_to_save:
            print("No results to save.")
            return

        standard_name = results_to_save.get("standard_name", "unknown_standard")
        # Sanitize standard_name for filename
        safe_standard_name = re.sub(r'[^\w\-_\. ]', '_', standard_name)
        timestamp = time.strftime("%Y%m%d_%H%M%S")
        
        filename_json = os.path.join(output_dir, f"{safe_standard_name}_full_results_{timestamp}.json")
        with open(filename_json, 'w', encoding='utf-8') as f:
            json.dump(results_to_save, f, indent=2, ensure_ascii=False)
        print(f"Full results saved to {filename_json}")

        # Also save the final report separately as Markdown
        final_report_text = results_to_save.get("final_report_output", {}).get("final_report")
        if final_report_text:
            filename_md = os.path.join(output_dir, f"{safe_standard_name}_final_report_{timestamp}.md")
            with open(filename_md, 'w', encoding='utf-8') as f:
                f.write(final_report_text)
            print(f"Final report saved to {filename_md}")


# --- Functions for recreating vector database safely ---
def safely_recreate_vector_database(documents):
    """Create a new vector database from document chunks with proper handling of locked files."""
    embeddings = OpenAIEmbeddings(model=config.EMBEDDING_MODEL)

    # Create a new directory with a unique name to avoid conflicts
    timestamp = time.strftime("%Y%m%d_%H%M%S")
    random_str = ''.join(random.choices(string.ascii_lowercase + string.digits, k=6))
    # Base directory for all DB versions
    base_db_parent_dir = "aaoifi_vector_db_versions"
    os.makedirs(base_db_parent_dir, exist_ok=True)
    new_db_directory = os.path.join(base_db_parent_dir, f"db_{timestamp}_{random_str}")

    print(f"Creating new database in directory: {new_db_directory}")
    os.makedirs(new_db_directory, exist_ok=True)

    client = chromadb.PersistentClient(path=new_db_directory)
    # Always create a new collection in a new DB path
    collection = client.create_collection(name=config.COLLECTION_NAME)
    
    batch_size = 100 # As used in create_vector_database
    num_batches = (len(documents) + batch_size - 1) // batch_size

    for i in range(0, len(documents), batch_size):
        batch_documents = documents[i:i+batch_size]
        current_batch_num = (i // batch_size) + 1
        print(f"Processing batch {current_batch_num}/{num_batches} for new vector database...")

        texts = [doc["content"] for doc in batch_documents]
        # Use more robust IDs tied to document source and chunk
        ids = [f"{doc['metadata']['source']}_chunk_{doc['metadata']['chunk_id']}" for doc in batch_documents]
        # Check for ID uniqueness within the batch (essential for Chroma)
        if len(ids) != len(set(ids)):
            print(f"Warning: Duplicate IDs generated in batch {current_batch_num}. This may cause issues.")
            # Simple fix: append index within batch to ensure uniqueness for this add op
            ids = [f"{id_}_{j}" for j, id_ in enumerate(ids)]

        metadatas = [doc["metadata"] for doc in batch_documents]

        if not texts:
            print(f"Skipping empty batch {current_batch_num}.")
            continue
        try:
            embeds = embeddings.embed_documents(texts)
            collection.add(embeddings=embeds, documents=texts, ids=ids, metadatas=metadatas)
        except Exception as e:
            print(f"Error processing batch {current_batch_num} starting at index {i}: {str(e)}")
            continue

    print(f"Created new vector database with consistent embedding dimensions in '{new_db_directory}'")
    
    # Update the global config to point to the new directory for the current session
    config.DB_DIRECTORY = new_db_directory
    print(f"Updated global config.DB_DIRECTORY to: {config.DB_DIRECTORY}")
    
    return client

def process_pdfs_safe():
    """Main function to process PDFs and create vector database safely."""
    PDF_FOLDER = config.PDF_FOLDER
    if not os.path.exists(PDF_FOLDER):
        print(f"Error: The PDF folder '{PDF_FOLDER}' does not exist.")
        return None
    if not os.listdir(PDF_FOLDER):
        print(f"The PDF folder '{PDF_FOLDER}' is empty. Please add PDF files to process.")
        return None

    all_documents = []
    pdf_files = [f for f in os.listdir(PDF_FOLDER) if f.lower().endswith('.pdf')]
    if not pdf_files:
        print(f"No PDF files found in '{PDF_FOLDER}'.")
        return None

    print(f"Found {len(pdf_files)} PDF files.")
    for pdf_file in pdf_files:
        pdf_path = os.path.join(PDF_FOLDER, pdf_file)
        standard_name = os.path.splitext(pdf_file)[0]
        print(f"Processing {pdf_file}...")
        raw_text = extract_text_from_pdf(pdf_path)
        cleaned_text = clean_text(raw_text)
        chunks = split_text_into_chunks(cleaned_text, standard_name)
        all_documents.extend(chunks)
        print(f"Extracted {len(chunks)} chunks from {pdf_file}")

    if not all_documents:
        print("No documents were extracted from PDFs. Cannot create vector database.")
        return None
        
    print(f"Total chunks extracted: {len(all_documents)}")
    print("Creating/Recreating vector database safely...")
    client = safely_recreate_vector_database(all_documents) # This updates config.DB_DIRECTORY
    print(f"Vector database operations completed. Current DB directory: '{config.DB_DIRECTORY}'")
    return client

# --- START OF NEW DEMONSTRATION CODE ---
"""
AAOIFI Standards Enhancement System Demo

This script demonstrates how to use the multi-agent architecture to:
1.  Select an AAOIFI standard
2.  Review and extract key elements using ReviewAgent
3.  Propose AI-driven modifications/enhancements using EnhancementAgent
4.  Validate proposed changes based on Shariah principles using ValidationAgent
5.  Generate a comprehensive report using FinalReportAgent
6.  Optionally, use additional agents for visualization and feedback simulation.
"""

class DemoRunner:
    """Class to run a complete demonstration of the AAOIFI Standards Enhancement System."""
    
    def __init__(self, system: AAOIFIStandardsEnhancementSystem):
        self.system = system
        self.selected_standard_name: Optional[str] = None
        self.current_demo_results: Dict[str, Any] = {} # Stores results for the current demo run
        
    def _get_excerpt(self, text: Optional[str], max_length: int = 300) -> str:
        """Get an excerpt of text with maximum length."""
        if not text:
            return "N/A"
        text = str(text) # Ensure it's a string
        if len(text) <= max_length:
            return text
        return text[:max_length] + "..."

    def list_and_select_standard(self) -> bool:
        """List all available standards and allow selection."""
        print("\n" + "="*50)
        print("AAOIFI STANDARDS ENHANCEMENT SYSTEM DEMONSTRATION")
        print("="*50)
        
        print("\nAvailable standards:")
        standards = self.system.list_available_standards()
        
        if not standards or any(s.startswith("Error:") or s.startswith("No standards found") for s in standards):
            print("No standards available or error listing standards.")
            print("Please ensure PDFs are processed and a vector database exists.")
            if standards: print(f"Details: {standards[0]}")
            return False
            
        for i, standard_name in enumerate(standards):
            print(f"{i+1}. {standard_name}")
        
        while True:
            try:
                choice_str = input(f"\nSelect a standard to process (enter number 1-{len(standards)}): ")
                if not choice_str: continue # Handle empty input
                choice = int(choice_str)
                if 1 <= choice <= len(standards):
                    self.selected_standard_name = standards[choice-1]
                    print(f"\nSelected standard: {self.selected_standard_name}")
                    self.current_demo_results = {"standard_name": self.selected_standard_name} # Reset for new selection
                    return True
                else:
                    print(f"Invalid selection. Please enter a number between 1 and {len(standards)}.")
            except ValueError:
                print("Invalid input. Please enter a number.")
    
    def run_review_phase(self) -> bool:
        """Run the review phase and display results."""
        if not self.selected_standard_name:
            print("Error: No standard selected for review phase.")
            return False

        print("\n" + "-"*50)
        print("PHASE 1: STANDARD REVIEW AND ANALYSIS (ReviewAgent)")
        print("-"*50)
        
        print(f"\nRetrieving content for standard: {self.selected_standard_name}...")
        content = self.system.db_manager.get_standard_content(self.selected_standard_name)
        if content.startswith("Error:") or content.startswith("No content found"):
            print(f"  Error retrieving content: {content}")
            self.current_demo_results["review_error"] = content
            return False
        
        standard_document = StandardDocument(name=self.selected_standard_name, content=content)
        
        print(f"Reviewing standard using ReviewAgent...")
        start_time = time.time()
        review_output = self.system.review_agent.execute(standard_document)
        self.current_demo_results["review_output"] = review_output
        elapsed_time = time.time() - start_time
        print(f"ReviewAgent completed in {elapsed_time:.2f} seconds.")
        
        print("\nREVIEW SUMMARY:")
        print(f"- Standard: {review_output.get('standard_name', 'N/A')}")
        print(f"\nCore Principles (excerpt):\n{self._get_excerpt(review_output.get('core_principles'))}")
        print(f"\nKey Definitions (excerpt):\n{self._get_excerpt(review_output.get('key_definitions'))}")
        # print(f"\nFull Review Text (excerpt):\n{self._get_excerpt(review_output.get('review_result'), 500)}")
        return True
    
    def run_enhancement_phase(self) -> bool:
        """Run the enhancement phase and display results."""
        if "review_output" not in self.current_demo_results:
            print("Error: Review phase must complete successfully before enhancement.")
            return False

        print("\n" + "-"*50)
        print("PHASE 2: AI-DRIVEN ENHANCEMENT PROPOSALS (EnhancementAgent)")
        print("-"*50)
        
        review_output = self.current_demo_results["review_output"]
        print(f"\nGenerating enhancement proposals for: {self.selected_standard_name} using EnhancementAgent...")
        
        start_time = time.time()
        enhancement_output = self.system.enhancement_agent.execute(review_output) # Pass full review dict
        self.current_demo_results["enhancement_output"] = enhancement_output
        elapsed_time = time.time() - start_time
        print(f"EnhancementAgent completed in {elapsed_time:.2f} seconds.")
        
        print("\nENHANCEMENT PROPOSALS (excerpt):")
        print(self._get_excerpt(enhancement_output.get("enhancement_proposals"), 500))
        return True

    def run_validation_phase(self) -> bool:
        """Run the validation phase and display results."""
        if "enhancement_output" not in self.current_demo_results or \
           "review_output" not in self.current_demo_results:
            print("Error: Review and Enhancement phases must complete successfully before validation.")
            return False

        print("\n" + "-"*50)
        print("PHASE 3: SHARIAH COMPLIANCE VALIDATION (ValidationAgent)")
        print("-"*50)

        review_output = self.current_demo_results["review_output"]
        enhancement_output = self.current_demo_results["enhancement_output"]
        
        print(f"\nValidating proposed enhancements for: {self.selected_standard_name} using ValidationAgent...")
        start_time = time.time()
        validation_output = self.system.validation_agent.execute(enhancement_output, review_output)
        self.current_demo_results["validation_output"] = validation_output
        elapsed_time = time.time() - start_time
        print(f"ValidationAgent completed in {elapsed_time:.2f} seconds.")
        
        print("\nVALIDATION RESULTS (excerpt):")
        print(self._get_excerpt(validation_output.get("validation_result"), 500))
        return True
    
    def run_report_generation_phase(self) -> bool:
        """Generate the final comprehensive report."""
        if "validation_output" not in self.current_demo_results: # Implies previous phases also ran
            print("Error: All previous phases (Review, Enhance, Validate) must complete before report generation.")
            return False

        print("\n" + "-"*50)
        print("PHASE 4: COMPREHENSIVE REPORT GENERATION (FinalReportAgent)")
        print("-"*50)
        
        print(f"\nGenerating comprehensive report for: {self.selected_standard_name} using FinalReportAgent...")
        
        report_input_data = {
            "standard_name": self.selected_standard_name,
            "review_text": self.current_demo_results.get("review_output", {}).get("review_result", "N/A"),
            "enhancements_text": self.current_demo_results.get("enhancement_output", {}).get("enhancement_proposals", "N/A"),
            "validation_text": self.current_demo_results.get("validation_output", {}).get("validation_result", "N/A")
        }
        
        start_time = time.time()
        final_report_output = self.system.report_agent.execute(report_input_data)
        self.current_demo_results["final_report_output"] = final_report_output
        elapsed_time = time.time() - start_time
        print(f"FinalReportAgent completed in {elapsed_time:.2f} seconds.")
        
        print("\nFINAL REPORT (excerpt):")
        print(self._get_excerpt(final_report_output.get("final_report"), 600))
        return True

    def save_demo_results(self):
        """Save all results from the demonstration."""
        if not self.current_demo_results or not self.selected_standard_name:
            print("No results to save or standard not selected.")
            return

        print("\n" + "-"*50)
        print("SAVING DEMONSTRATION RESULTS")
        print("-"*50)
        
        # Use the system's save_results method, passing the demo's collected results
        self.system.save_results(self.current_demo_results) # It handles directory creation and naming
    
    def run_complete_demo(self):
        """Run the complete demonstration process."""
        if not self.list_and_select_standard():
            print("Demo terminated: Standard selection failed or was aborted.")
            return
        
        if not self.run_review_phase():
            print("Demo terminated: Review phase failed.")
            return
        
        if not self.run_enhancement_phase():
            print("Demo terminated: Enhancement phase failed.")
            return
        
        if not self.run_validation_phase():
            print("Demo terminated: Validation phase failed.")
            return
        
        if not self.run_report_generation_phase():
            print("Demo terminated: Report generation phase failed.")
            return
        
        self.save_demo_results()
        
        print("\n" + "="*50)
        print(f"DEMONSTRATION COMPLETED SUCCESSFULLY FOR STANDARD: {self.selected_standard_name}")
        print("="*50)
        print(f"All results and the final report have been saved to the '{config.OUTPUT_DIR}' directory.")

# --- Additional Agents for Enhanced Demo ---

class VisualizationAgent(BaseAgent):
    """Agent responsible for generating textual descriptions for visualizations."""
    
    def __init__(self):
        super().__init__(
            name="VisualizationAgent",
            description="Creates textual summaries suitable for generating visual representations of standard changes.",
            model_name=Config.GPT35_MODEL # Faster model for summarization
        )
        
        self.system_prompt = """
        You are a data visualization assistant. Your task is to process the provided summaries of AAOIFI standard reviews,
        enhancement proposals, and validation results, and then generate concise textual descriptions that could be used
        to create visual elements (like tables or charts).

        Focus on:
        1.  A summary table of key proposed changes: Section | Original Concept | Proposed Change | Benefit.
        2.  A Shariah compliance assessment summary: Proposal Area | Compliance Status | Key Shariah Considerations.
        3.  A benefits overview: Key Enhancement | Primary Benefit to Stakeholders.

        Present this information clearly using Markdown tables or structured lists.
        """
    
    def _get_excerpt(self, text: Optional[str], max_length: int = 300) -> str: # Helper from DemoRunner
        if not text: return "N/A"
        text = str(text)
        if len(text) <= max_length: return text
        return text[:max_length] + "..."

    def execute(self, demo_results: Dict[str, Any]) -> Dict[str, Any]:
        standard_name = demo_results.get('standard_name', 'N/A')
        review_text = self._get_excerpt(demo_results.get('review_output', {}).get('review_result', 'N/A'), 1000)
        enhancements_text = self._get_excerpt(demo_results.get('enhancement_output', {}).get('enhancement_proposals', 'N/A'), 1000)
        validation_text = self._get_excerpt(demo_results.get('validation_output', {}).get('validation_result', 'N/A'), 1000)

        prompt = ChatPromptTemplate.from_messages([
            SystemMessage(content=self.system_prompt),
            HumanMessage(content=f"""
            Standard Name: {standard_name}
            
            Summary of Standard Review:
            {review_text}
            
            Summary of Proposed Enhancements:
            {enhancements_text}
            
            Summary of Validation Results:
            {validation_text}
            
            Please generate textual descriptions suitable for visualization based on the above.
            """)
        ])
        
        chain = LLMChain(llm=self.llm, prompt=prompt)
        visualization_text_data = chain.run({})
        
        return {
            "standard_name": standard_name,
            "visualization_text_data": visualization_text_data
        }

class FeedbackAgent(BaseAgent):
    """Agent for collecting and analyzing simulated feedback on proposed changes."""
    
    def __init__(self):
        super().__init__(
            name="FeedbackAgent",
            description="Processes simulated feedback on proposed standard changes and generates insights.",
            model_name=Config.GPT35_MODEL
        )
        
        self.system_prompt = """
        You are an expert in qualitative feedback analysis for financial standards.
        Given a set of simulated feedback entries on proposed changes to an AAOIFI standard,
        your task is to:

        1.  Summarize the overall sentiment (Positive, Negative, Mixed).
        2.  Identify 2-3 key themes or concerns raised by stakeholders.
        3.  Highlight 2-3 constructive suggestions for further improvement, if any.
        4.  Note any areas of strong consensus or significant disagreement.

        Provide a concise analysis.
        """
    
    def execute(self, standard_name: str, simulated_feedback_list: List[Dict[str, Any]]) -> Dict[str, Any]:
        formatted_feedback = "\n\n".join([
            f"Feedback Entry #{i+1}:\nStakeholder Role: {item.get('role', 'N/A')}\nRating: {item.get('rating', 'N/A')}/5\nComment: {item.get('comment', 'N/A')}"
            for i, item in enumerate(simulated_feedback_list)
        ])
        
        prompt = ChatPromptTemplate.from_messages([
            SystemMessage(content=self.system_prompt),
            HumanMessage(content=f"""
            Standard Name: {standard_name}
            
            Simulated Stakeholder Feedback:
            {formatted_feedback}
            
            Please analyze this feedback.
            """)
        ])
        
        chain = LLMChain(llm=self.llm, prompt=prompt)
        analysis_result = chain.run({})
        
        avg_rating = sum(item.get('rating', 0) for item in simulated_feedback_list) / len(simulated_feedback_list) if simulated_feedback_list else 0
        
        return {
            "standard_name": standard_name,
            "feedback_count": len(simulated_feedback_list),
            "average_simulated_rating": f"{avg_rating:.2f}/5.00",
            "feedback_analysis_summary": analysis_result
        }

class EnhancedDemoRunner(DemoRunner):
    """Enhanced demo runner with visualization and feedback capabilities."""
    
    def __init__(self, system: AAOIFIStandardsEnhancementSystem):
        super().__init__(system)
        self.visualization_agent = VisualizationAgent()
        self.feedback_agent = FeedbackAgent()
        
    def run_visualization_phase(self) -> bool:
        """Generate textual descriptions for visualizations based on the results."""
        if not self.current_demo_results.get("final_report_output"): # Check if prior phases are done
            print("Error: Core demo phases must complete before visualization.")
            return False

        print("\n" + "-"*50)
        print("PHASE 5: GENERATING VISUALIZATION DATA (VisualizationAgent)")
        print("-"*50)
        
        print(f"\nGenerating visualization text for: {self.selected_standard_name} using VisualizationAgent...")
        start_time = time.time()
        visualization_output = self.visualization_agent.execute(self.current_demo_results)
        self.current_demo_results["visualization_output"] = visualization_output
        elapsed_time = time.time() - start_time
        print(f"VisualizationAgent completed in {elapsed_time:.2f} seconds.")
        
        print("\nVISUALIZATION TEXT DATA (excerpt):")
        print(self._get_excerpt(visualization_output.get("visualization_text_data"), 500))
        return True
    
    def run_simulated_feedback_phase(self) -> bool:
        """Simulate stakeholder feedback and analyze it."""
        if not self.current_demo_results.get("final_report_output"):
            print("Error: Core demo phases must complete before feedback simulation.")
            return False

        print("\n" + "-"*50)
        print("PHASE 6: SIMULATED FEEDBACK ANALYSIS (FeedbackAgent)")
        print("-"*50)
        
        print(f"\nSimulating stakeholder feedback for: {self.selected_standard_name}...")
        simulated_feedback = [
            {"role": "Shariah Scholar", "rating": 4, "comment": "The proposals are largely compliant, but clarification is needed on the application of 'gharar yaseer' in proposed digital contracts."},
            {"role": "Banker", "rating": 5, "comment": "These changes will significantly improve operational efficiency and reduce ambiguity in sukuk issuance."},
            {"role": "Regulator", "rating": 3, "comment": "While the intent is good, the proposed technological integrations might pose supervisory challenges. We need more detailed risk management guidelines."},
            {"role": "Academic", "rating": 4, "comment": "A solid step forward. I suggest including references to contemporary research on fintech in Islamic finance for a more robust theoretical underpinning."}
        ]
        print(f"Generated {len(simulated_feedback)} simulated feedback entries.")

        print(f"Analyzing simulated feedback using FeedbackAgent...")
        start_time = time.time()
        feedback_analysis_output = self.feedback_agent.execute(self.selected_standard_name, simulated_feedback)
        self.current_demo_results["feedback_analysis_output"] = feedback_analysis_output
        elapsed_time = time.time() - start_time
        print(f"FeedbackAgent completed in {elapsed_time:.2f} seconds.")

        print("\nSIMULATED FEEDBACK ANALYSIS (excerpt):")
        print(self._get_excerpt(feedback_analysis_output.get("feedback_analysis_summary"), 500))
        return True

    def run_complete_enhanced_demo(self):
        """Run the complete enhanced demonstration process."""
        if not self.list_and_select_standard():
            print("Enhanced Demo terminated: Standard selection failed.")
            return
        
        if not self.run_review_phase():
            print("Enhanced Demo terminated: Review phase failed.")
            return
        
        if not self.run_enhancement_phase():
            print("Enhanced Demo terminated: Enhancement phase failed.")
            return
        
        if not self.run_validation_phase():
            print("Enhanced Demo terminated: Validation phase failed.")
            return
        
        if not self.run_report_generation_phase():
            print("Enhanced Demo terminated: Report generation phase failed.")
            return
        
        # Enhanced Steps
        if not self.run_visualization_phase():
            print("Enhanced Demo warning: Visualization phase failed, but core demo completed.")
        
        if not self.run_simulated_feedback_phase():
            print("Enhanced Demo warning: Feedback simulation phase failed, but core demo completed.")
            
        self.save_demo_results()
        
        print("\n" + "="*50)
        print(f"ENHANCED DEMONSTRATION COMPLETED SUCCESSFULLY FOR STANDARD: {self.selected_standard_name}")
        print("="*50)
        print(f"All results, including enhanced analyses, have been saved to the '{config.OUTPUT_DIR}' directory.")


# Main execution block for the demo
def main_demo():
    """Main function to run the demonstration."""
    print("Initializing AAOIFI Standards Enhancement System for Demo...")
    
    # Step 1: Ensure Vector DB is populated (or try to populate it)
    # Try to initialize system first. If it fails badly (e.g. DB dir issue), then try to process PDFs.
    try:
        system_check = AAOIFIStandardsEnhancementSystem()
        standards_check = system_check.list_available_standards()
        if not standards_check or any(s.startswith("Error:") or s.startswith("No standards found") for s in standards_check):
            print("\nNo standards readily available or error accessing existing DB.")
            run_pdf_processing = input("Attempt to process PDFs in 'pdf_eng' folder to create/recreate database? (yes/no): ").strip().lower()
            if run_pdf_processing == 'yes':
                print("Processing PDFs using safe recreate method...")
                process_pdfs_safe() # This updates config.DB_DIRECTORY
                print("Re-initializing system with the new/updated database...")
                # System will be initialized below with the potentially new config.DB_DIRECTORY
            else:
                print("PDF processing skipped. Demo may not function if no standards are available.")
        else:
            print(f"Found existing standards: {standards_check[:3]}...") # Print first few
    except Exception as e:
        print(f"Initial system/DB check failed: {e}")
        run_pdf_processing = input("Attempt to process PDFs in 'pdf_eng' folder to create/recreate database? (yes/no): ").strip().lower()
        if run_pdf_processing == 'yes':
            print("Processing PDFs using safe recreate method...")
            process_pdfs_safe() # This updates config.DB_DIRECTORY
            print("Re-initializing system with the new/updated database...")
        else:
            print("PDF processing skipped. Demo cannot continue without a functional database.")
            return

    # Initialize the main system for the demo (picks up current config.DB_DIRECTORY)
    try:
        system = AAOIFIStandardsEnhancementSystem()
    except Exception as e:
        print(f"Fatal error initializing AAOIFIStandardsEnhancementSystem: {e}")
        print("Please check your database setup and OpenAI API key.")
        return

    # Choose which demo to run
    print("\nSelect Demo Type:")
    print("1. Basic Demo (Review, Enhance, Validate, Report)")
    print("2. Enhanced Demo (includes Visualization and Feedback Analysis)")
    
    while True:
        try:
            choice_str = input("Enter your choice (1 or 2, or 'exit'): ").strip()
            if choice_str.lower() == 'exit':
                print("Exiting demo.")
                break
            if not choice_str: continue

            choice = int(choice_str)
            if choice == 1:
                demo = DemoRunner(system)
                demo.run_complete_demo()
                break
            elif choice == 2:
                demo = EnhancedDemoRunner(system)
                demo.run_complete_enhanced_demo()
                break
            else:
                print("Invalid selection. Please enter 1 or 2.")
        except ValueError:
            print("Invalid input. Please enter a number (1 or 2) or 'exit'.")
        except Exception as e:
            print(f"An unexpected error occurred during demo execution: {e}")
            # traceback.print_exc() # For debugging
            break

if __name__ == "__main__":
    # Ensure OPENAI_API_KEY is set
    if not os.environ.get("OPENAI_API_KEY") or "sk-proj-" not in os.environ.get("OPENAI_API_KEY"): # Basic check
        print("Error: OPENAI_API_KEY is not set or appears invalid in the script.")
        print("Please set it near the top of the script (line 24 approx).")
        # You might want to exit here if the key is critical for all operations
        # exit() # Uncomment if you want to force exit

    # Check if pdf_eng folder exists and has PDFs, offer to run process_pdfs_safe if empty
    pdf_folder = Config.PDF_FOLDER
    if not os.path.exists(pdf_folder):
        print(f"PDF folder '{pdf_folder}' does not exist. Please create it and add AAOIFI standard PDFs.")
    elif not [f for f in os.listdir(pdf_folder) if f.lower().endswith('.pdf')]:
        print(f"PDF folder '{pdf_folder}' is empty. Please add AAOIFI standard PDFs to process.")
    
    # Run the main demonstration logic
    main_demo()

Initializing AAOIFI Standards Enhancement System for Demo...
Initializing VectorDBManager with database directory: aaoifi_vector_db_20250510_171304_gqfbl3
Successfully created PersistentClient for database at: aaoifi_vector_db_20250510_171304_gqfbl3
Successfully accessed/created collection: aaoifi_standards
Found existing standards: ['AAOIFI-19th-SB-Conf.-Arabic-Agenda-V9', 'AAOIFI-20th-SB-Conf.-Arabic-Agenda-V5LR', 'AAOIFI-Conceptual-Framework-for-Financial-Reporting-Revised-2020-Final-clean']...
Initializing VectorDBManager with database directory: aaoifi_vector_db_20250510_171304_gqfbl3
Successfully created PersistentClient for database at: aaoifi_vector_db_20250510_171304_gqfbl3
Successfully accessed/created collection: aaoifi_standards

Select Demo Type:
1. Basic Demo (Review, Enhance, Validate, Report)
2. Enhanced Demo (includes Visualization and Feedback Analysis)

AAOIFI STANDARDS ENHANCEMENT SYSTEM DEMONSTRATION

Available standards:
1. AAOIFI-19th-SB-Conf.-Arabic-Agenda-V9
2.

In [27]:
import logging
import os
import json
import re
import time
import uuid
from typing import Dict, List, Any

# Add these imports which are referenced but not imported in the original code
import chromadb
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import SystemMessage, HumanMessage
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, Markdown

# Configure logger
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(),
        logging.FileHandler("aaoifi_standards_system.log")
    ]
)

logger = logging.getLogger("AAOIFI_MAS")

# Define the Config class that's referenced throughout the code
class Config:
    OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY", "")
    MODEL_NAME = "gpt-4"
    EMBEDDING_MODEL = "text-embedding-ada-002"
    DB_DIRECTORY = "./vector_db"
    COLLECTION_NAME = "aaoifi_standards"
    OUTPUT_DIR = "./output"

# Define the StandardDocument class referenced in the code
class StandardDocument:
    def __init__(self, name: str, content: str):
        self.name = name
        self.content = content

# Define the BaseAgent class that other agents inherit from
class BaseAgent:
    def __init__(self, name: str, description: str, agent_type: str):
        self.name = name
        self.description = description
        self.agent_type = agent_type
        self.system_prompt = ""
    
    def _run_chain(self, messages: List):
        """Run the LLM chain with the given messages."""
        # In a real implementation, this would use the LLM
        # For now, just return a placeholder
        return "This is a placeholder for LLM response"
    
    def log_execution(self, input_name: str, output_name: str, start_time: float):
        """Log the execution details."""
        execution_time = time.time() - start_time
        logger.info(f"{self.name} processed {input_name} into {output_name} in {execution_time:.2f} seconds")

# Define the DocumentProcessor class referenced in the code
class DocumentProcessor:
    @staticmethod
    def split_text_into_chunks(text: str, doc_name: str, chunk_size: int = 1000):
        """Split text into chunks with metadata."""
        # Simple chunking by character count
        chunks = []
        for i in range(0, len(text), chunk_size):
            chunk_text = text[i:i+chunk_size]
            chunks.append({
                "content": chunk_text,
                "metadata": {
                    "source": doc_name,
                    "chunk": i // chunk_size
                }
            })
        return chunks

In [ ]:
!pip freeze> requirements.txt

In [36]:
import os
import re
import json
import numpy as np
from typing import List, Dict, Any, Optional
from datetime import datetime
import pandas as pd

class IslamicAccountingStandard:
    """Base class for Islamic accounting standards."""
    def __init__(self, name: str, description: str):
        self.name = name
        self.description = description
        self.use_cases = []
    
    def add_use_case(self, use_case):
        """Add a use case to this standard."""
        self.use_cases.append(use_case)
        
    def get_use_cases(self):
        """Get all use cases for this standard."""
        return self.use_cases


class AccountingUseCase:
    """Class representing an accounting use case with scenario and solution."""
    def __init__(self, 
                 category: str, 
                 subcategory: str,
                 scenario: str, 
                 question: str,
                 correct_solution: str,
                 metadata: Optional[Dict] = None):
        self.category = category
        self.subcategory = subcategory
        self.scenario = scenario
        self.question = question
        self.correct_solution = correct_solution
        self.metadata = metadata or {}
        self.solution_components = self._extract_solution_components()
    
    def _extract_solution_components(self):
        """Extract key components from the correct solution."""
        components = {}
        
        # Extract journal entries from the solution
        journal_entries = []
        dr_cr_pattern = r'Dr\.\s+([^USD]+)\s+USD\s+([\d,]+)[\r\n]+Cr\.\s+([^USD]+)\s+USD\s+([\d,]+)'
        matches = re.findall(dr_cr_pattern, self.correct_solution)
        
        for match in matches:
            debit_account = match[0].strip()
            debit_amount = float(match[1].replace(',', ''))
            credit_account = match[2].strip()
            credit_amount = float(match[3].replace(',', ''))
            
            journal_entries.append({
                'debit_account': debit_account,
                'debit_amount': debit_amount,
                'credit_account': credit_account,
                'credit_amount': credit_amount
            })
        
        components['journal_entries'] = journal_entries
        
        # Extract calculations and values
        amount_pattern = r'([A-Za-z\s]+)[:=]\s*\$?USD?\s*([\d,]+)'
        calculations = re.findall(amount_pattern, self.correct_solution)
        for calc in calculations:
            key = calc[0].strip().lower().replace(' ', '_')
            value = float(calc[1].replace(',', ''))
            components[key] = value
            
        return components
    
    def to_dict(self):
        """Convert the use case to a dictionary."""
        return {
            'category': self.category,
            'subcategory': self.subcategory,
            'scenario': self.scenario,
            'question': self.question,
            'correct_solution': self.correct_solution,
            'metadata': self.metadata,
            'solution_components': self.solution_components
        }


class IjarahAccountingAgent:
    """Agent for handling Ijarah accounting use cases."""
    def __init__(self):
        self.name = "Ijarah Accounting Expert"
        self.description = "Specialized agent for Islamic finance Ijarah accounting scenarios"
        self.standards = {}
        self.use_cases = []
        
    def add_standard(self, standard: IslamicAccountingStandard):
        """Add an accounting standard to the agent's knowledge."""
        self.standards[standard.name] = standard
        
    def add_use_case(self, use_case: AccountingUseCase, standard_name: Optional[str] = None):
        """Add a use case to the agent and optionally to a specific standard."""
        self.use_cases.append(use_case)
        if standard_name and standard_name in self.standards:
            self.standards[standard_name].add_use_case(use_case)
            
    def load_use_cases_from_json(self, json_file_path: str):
        """Load use cases from a JSON file."""
        try:
            with open(json_file_path, 'r') as f:
                data = json.load(f)
                
            for item in data:
                use_case = AccountingUseCase(
                    category=item.get('category', ''),
                    subcategory=item.get('subcategory', ''),
                    scenario=item.get('scenario', ''),
                    question=item.get('question', ''),
                    correct_solution=item.get('correct_solution', ''),
                    metadata=item.get('metadata', {})
                )
                self.add_use_case(use_case, item.get('standard', None))
                
            return True
        except Exception as e:
            print(f"Error loading use cases: {e}")
            return False
    
    def save_use_cases_to_json(self, json_file_path: str):
        """Save all use cases to a JSON file."""
        try:
            data = [use_case.to_dict() for use_case in self.use_cases]
            with open(json_file_path, 'w') as f:
                json.dump(data, f, indent=2)
            return True
        except Exception as e:
            print(f"Error saving use cases: {e}")
            return False
            
    def find_similar_use_case(self, query: str, top_n: int = 1):
        """Find use cases similar to the query using simple keyword matching."""
        # This is a simple implementation; in a real system, you would use embeddings
        query_words = set(query.lower().split())
        scored_cases = []
        
        for use_case in self.use_cases:
            # Combine all text fields for searching
            use_case_text = f"{use_case.category} {use_case.subcategory} {use_case.scenario} {use_case.question}"
            use_case_words = set(use_case_text.lower().split())
            
            # Calculate simple overlap score
            overlap = len(query_words.intersection(use_case_words))
            if overlap > 0:
                scored_cases.append((use_case, overlap))
        
        # Sort by score (highest first) and return top_n
        scored_cases.sort(key=lambda x: x[1], reverse=True)
        return [case for case, score in scored_cases[:top_n]]


class IjarahMBTCalculator:
    """
    Calculator for Ijarah Muntahia Bittamleek (MBT) accounting calculations.
    Handles calculations for both lessor and lessee accounting.
    """
    
    @staticmethod
    def calculate_rou_asset(asset_cost: float, 
                          import_tax: float,
                          freight: float,
                          terminal_value: float) -> float:
        """
        Calculate the Right of Use (ROU) asset amount.
        
        Args:
            asset_cost: The base cost of the asset
            import_tax: Import tax paid on the asset
            freight: Freight charges for the asset
            terminal_value: Expected terminal/residual value
            
        Returns:
            The calculated ROU asset value
        """
        prime_cost = asset_cost + import_tax + freight
        rou = prime_cost - terminal_value
        return rou
    
    @staticmethod
    def calculate_deferred_ijarah_cost(total_rentals: float, 
                                      rou_value: float) -> float:
        """
        Calculate the deferred Ijarah cost.
        
        Args:
            total_rentals: Total rental payments over the Ijarah term
            rou_value: Value of the Right of Use asset
            
        Returns:
            The calculated deferred Ijarah cost
        """
        return total_rentals - rou_value
    
    @staticmethod
    def calculate_amortizable_amount(rou_value: float, 
                                   residual_value: float,
                                   purchase_price: float) -> float:
        """
        Calculate the amortizable amount.
        
        Args:
            rou_value: Value of the Right of Use asset
            residual_value: Expected residual value of the asset
            purchase_price: Price to purchase the asset at the end of term
            
        Returns:
            The calculated amortizable amount
        """
        terminal_value_difference = residual_value - purchase_price
        return rou_value - terminal_value_difference
    
    @staticmethod
    def calculate_annual_amortization(amortizable_amount: float, 
                                    ijarah_term_years: int) -> float:
        """
        Calculate the annual amortization amount.
        
        Args:
            amortizable_amount: Amount to be amortized
            ijarah_term_years: Duration of the Ijarah term in years
            
        Returns:
            The annual amortization amount
        """
        return amortizable_amount / ijarah_term_years
    
    @staticmethod
    def generate_lessee_initial_recognition_entry(rou_value: float, 
                                               deferred_cost: float,
                                               total_liability: float) -> Dict:
        """
        Generate journal entry for lessee's initial recognition.
        
        Args:
            rou_value: Value of the Right of Use asset
            deferred_cost: Deferred Ijarah cost
            total_liability: Total Ijarah liability
            
        Returns:
            Dictionary containing the journal entry details
        """
        return {
            "entry_type": "Initial Recognition (Lessee)",
            "debit_entries": [
                {"account": "Right of Use Asset (ROU)", "amount": round(rou_value, 2)},
                {"account": "Deferred Ijarah Cost", "amount": round(deferred_cost, 2)}
            ],
            "credit_entries": [
                {"account": "Ijarah Liability", "amount": round(total_liability, 2)}
            ]
        }
    
    @staticmethod
    def generate_lessee_rental_payment_entry(rental_amount: float) -> Dict:
        """
        Generate journal entry for lessee's rental payment.
        
        Args:
            rental_amount: Periodic rental payment amount
            
        Returns:
            Dictionary containing the journal entry details
        """
        return {
            "entry_type": "Rental Payment (Lessee)",
            "debit_entries": [
                {"account": "Ijarah Liability", "amount": round(rental_amount, 2)}
            ],
            "credit_entries": [
                {"account": "Cash/Bank", "amount": round(rental_amount, 2)}
            ]
        }
    
    @staticmethod
    def generate_lessee_amortization_entry(amortization_amount: float) -> Dict:
        """
        Generate journal entry for lessee's amortization of ROU asset.
        
        Args:
            amortization_amount: Amount of amortization for the period
            
        Returns:
            Dictionary containing the journal entry details
        """
        return {
            "entry_type": "Amortization (Lessee)",
            "debit_entries": [
                {"account": "Ijarah Expense", "amount": round(amortization_amount, 2)}
            ],
            "credit_entries": [
                {"account": "Right of Use Asset (ROU)", "amount": round(amortization_amount, 2)}
            ]
        }


class IjarahUseCaseProcessor:
    """
    Processes Ijarah use cases and generates standardized solutions.
    """
    def __init__(self):
        self.calculator = IjarahMBTCalculator()
        
    def process_ijarah_mbt_lessee_case(self, use_case_data: Dict) -> Dict:
        """
        Process an Ijarah MBT use case for lessee accounting.
        
        Args:
            use_case_data: Dictionary containing the use case details
            
        Returns:
            Dictionary with the processed solution
        """
        # Extract relevant parameters from the use case
        asset_cost = use_case_data.get('asset_cost')
        import_tax = use_case_data.get('import_tax', 0)
        freight = use_case_data.get('freight', 0)
        ijarah_term = use_case_data.get('ijarah_term')
        residual_value = use_case_data.get('residual_value', 0)
        purchase_price = use_case_data.get('purchase_price', 0)
        annual_rental = use_case_data.get('annual_rental')
        
        # Calculate key values
        prime_cost = asset_cost + import_tax + freight
        rou_value = self.calculator.calculate_rou_asset(
            asset_cost, import_tax, freight, purchase_price)
        
        total_rentals = annual_rental * ijarah_term
        deferred_cost = self.calculator.calculate_deferred_ijarah_cost(
            total_rentals, rou_value)
        
        terminal_value_difference = residual_value - purchase_price
        amortizable_amount = self.calculator.calculate_amortizable_amount(
            rou_value, residual_value, purchase_price)
        
        # Generate journal entries
        initial_entry = self.calculator.generate_lessee_initial_recognition_entry(
            rou_value, deferred_cost, total_rentals)
        
        # Format the solution
        solution = {
            "calculations": {
                "prime_cost": prime_cost,
                "right_of_use_asset": rou_value,
                "total_rentals": total_rentals,
                "deferred_ijarah_cost": deferred_cost,
                "terminal_value_difference": terminal_value_difference,
                "amortizable_amount": amortizable_amount
            },
            "journal_entries": [initial_entry]
        }
        
        return solution
    
    def format_solution_markdown(self, solution: Dict) -> str:
        """
        Format the solution into a readable markdown string.
        
        Args:
            solution: Dictionary containing the solution details
            
        Returns:
            Formatted solution as markdown string
        """
        md = "## Solution\n\n"
        
        # Format calculations
        md += "### Calculations\n\n"
        calcs = solution.get('calculations', {})
        for key, value in calcs.items():
            md += f"**{key.replace('_', ' ').title()}**: USD {value:,.2f}\n"
        
        # Format journal entries
        md += "\n### Journal Entries\n\n"
        for entry in solution.get('journal_entries', []):
            md += f"**{entry.get('entry_type')}**:\n\n"
            
            for debit in entry.get('debit_entries', []):
                md += f"Dr. {debit.get('account')} USD {debit.get('amount'):,.2f}\n"
                
            for credit in entry.get('credit_entries', []):
                md += f"Cr. {credit.get('account')} USD {credit.get('amount'):,.2f}\n"
            
            md += "\n"
        
        return md


class IslamicFinanceAccountingSystem:
    """Main system for Islamic finance accounting use cases."""
    
    def __init__(self):
        self.ijarah_agent = IjarahAccountingAgent()
        self.use_case_processor = IjarahUseCaseProcessor()
        self.standards_db = {}
        self.initialize_system()
        
    def initialize_system(self):
        """Initialize the system with built-in standards and use cases."""
        # Create standard definitions
        aaoifi_fas_8 = IslamicAccountingStandard(
            name="AAOIFI FAS 8",
            description="Financial Accounting Standard 8: Ijarah and Ijarah Muntahia Bittamleek"
        )
        
        # Add standards to the system
        self.standards_db["AAOIFI FAS 8"] = aaoifi_fas_8
        self.ijarah_agent.add_standard(aaoifi_fas_8)
        
        # Add built-in use cases
        self._add_builtin_use_cases()
    
    def _add_builtin_use_cases(self):
        """Add built-in use cases to the system."""
        # Add the example use case from the problem statement
        ijarah_mbt_use_case = AccountingUseCase(
            category="Ijarah MBT Accounting",
            subcategory="Lessee's Books",
            scenario="""
            On 1 January 2019 Alpha Islamic bank (Lessee) entered into an Ijarah MBT arrangement with
            Super Generators for Ijarah of a heavy-duty generator purchase by Super Generators at a price
            of USD 450,000. Super Generators has also paid USD 12,000 as import tax and US 30,000 for
            freight charges. The Ijarah Term is 02 years and expected residual value at the end USD 5,000.
            At the end of Ijarah Term, it is highly likely that the option of transfer of ownership of the
            underlying asset to the lessee shall be exercised through purchase at a price of USD 3,000.
            Alpha Islamic Bank will amortize the 'right of use' on yearly basis and it is required to pay
            yearly rental of USD 300,000.
            """,
            question="""
            Provide the following accounting entry in the books of Alpha Islamic Bank:
            Initial Recognition at the time of commencement of Ijarah (using Underlying Asset Cost Method).
            """,
            correct_solution="""
            Initial Recognition at the Time of Commencement of Ijarah (1 January 2019):
            The cost of the underlying asset is the basis for recognizing the "Right of Use" (ROU) asset.
            
            Determine the Right-of-Use Asset (ROU)
            • Prime cost (Purchase + Import tax + Freight): = 450,000 + 12,000 + 30,000 = 492,000
            • Less: Terminal value (i.e., Purchase price of USD 3,000 to acquire ownership) = 492,000 − 3,000 = 489,000 (ROU)
            
            Add Deferred Ijarah Cost:
            • Total rentals over 2 years = 300,000 × 2 = 600,000
            • Less: Present value of ROU = 489,000
            • Deferred Ijarah Cost = 600,000 − 489,000 = 111,000
            
            Journal Entry:
            Dr. Right of Use Asset (ROU) USD 489,000
            Dr. Deferred Ijarah Cost USD 111,000
            Cr. Ijarah Liability USD 600,000
            
            Amortizable Amount Calculation
            Description USD
            Cost of ROU 489,000
            Less: Terminal value difference (Residual 5,000 − Purchase 3,000) 2,000
            Amortizable Amount 487,000
            We deduct this 2,000 since the Lessee is expected to gain ownership
            """,
            metadata={
                "asset_cost": 450000,
                "import_tax": 12000,
                "freight": 30000,
                "ijarah_term": 2,
                "residual_value": 5000,
                "purchase_price": 3000,
                "annual_rental": 300000,
                "standard": "AAOIFI FAS 8"
            }
        )
        
        self.ijarah_agent.add_use_case(ijarah_mbt_use_case, "AAOIFI FAS 8")
    
    def parse_user_query(self, query: str) -> Dict:
        """
        Parse a user query to extract relevant parameters.
        This is a simplified implementation that would be enhanced with NLP.
        
        Args:
            query: The user's query string
            
        Returns:
            Dictionary containing extracted parameters
        """
        params = {}
        
        # Extract numbers using regex
        numbers = re.findall(r'USD\s*([\d,]+)', query)
        numbers = [float(num.replace(',', '')) for num in numbers]
        
        # Look for specific keywords and associate with parameters
        if 'ijarah mbt' in query.lower():
            params['transaction_type'] = 'ijarah_mbt'
        
        if 'lessee' in query.lower():
            params['party'] = 'lessee'
        elif 'lessor' in query.lower():
            params['party'] = 'lessor'
            
        if 'initial recognition' in query.lower():
            params['entry_type'] = 'initial_recognition'
            
        if 'rental payment' in query.lower():
            params['entry_type'] = 'rental_payment'
            
        if 'amortization' in query.lower():
            params['entry_type'] = 'amortization'
            
        # Attempt to extract financial values based on context
        if len(numbers) >= 3:
            # This is very simplified and would need enhancement in a real system
            if len(numbers) >= 5:
                params['asset_cost'] = numbers[0]
            
        return params
    
    def generate_accounting_entry(self, use_case_data: Dict) -> Dict:
        """
        Generate accounting entries based on the use case data.
        
        Args:
            use_case_data: Dictionary with parameters for the use case
            
        Returns:
            Solution dictionary with calculations and journal entries
        """
        transaction_type = use_case_data.get('transaction_type', '')
        party = use_case_data.get('party', '')
        
        if transaction_type == 'ijarah_mbt' and party == 'lessee':
            solution = self.use_case_processor.process_ijarah_mbt_lessee_case(use_case_data)
            return solution
        
        # Handle other cases as needed
        return {"error": "Unsupported transaction type or party"}
    
    def find_matching_use_case(self, query: str) -> Optional[AccountingUseCase]:
        """
        Find a matching use case from the database based on the query.
        
        Args:
            query: The user's query string
            
        Returns:
            Matching use case if found, None otherwise
        """
        similar_cases = self.ijarah_agent.find_similar_use_case(query)
        if similar_cases:
            return similar_cases[0]
        return None
    
    def process_query(self, query: str) -> Dict:
        """
        Process a user query and generate a response.
        
        Args:
            query: The user's query string
            
        Returns:
            Dictionary with the response
        """
        # First try to find a matching use case
        matching_case = self.find_matching_use_case(query)
        if matching_case:
            # Use the matching case's metadata for processing
            solution = self.generate_accounting_entry(matching_case.metadata)
            formatted_solution = self.use_case_processor.format_solution_markdown(solution)
            
            return {
                "found_matching_case": True,
                "use_case": matching_case.to_dict(),
                "generated_solution": solution,
                "formatted_solution": formatted_solution,
                "correct_solution": matching_case.correct_solution
            }
        
        # If no matching case, try to parse the query and generate a solution
        params = self.parse_user_query(query)
        if params:
            solution = self.generate_accounting_entry(params)
            formatted_solution = self.use_case_processor.format_solution_markdown(solution)
            
            return {
                "found_matching_case": False,
                "parsed_params": params,
                "generated_solution": solution,
                "formatted_solution": formatted_solution
            }
        
        return {"error": "Could not parse query or find matching use case"}


# Example usage
if __name__ == "__main__":
    # Initialize the system
    islamic_finance_system = IslamicFinanceAccountingSystem()
    
    # Example query
    query = """
    I need the initial recognition entry for an Ijarah MBT arrangement where the lessee (Islamic bank)
    is leasing a generator purchased at USD 450,000 with USD 12,000 import tax and USD 30,000 freight.
    The Ijarah term is 2 years with yearly rental of USD 300,000 and expected residual value of USD 5,000.
    The lessee will purchase the asset at the end for USD 3,000.
    """
    
    # Process the query
    result = islamic_finance_system.process_query(query)
    
    # Print the result
    if "formatted_solution" in result:
        print(result["formatted_solution"])
    elif "error" in result:
        print(f"Error: {result['error']}")

## Solution

### Calculations


### Journal Entries




In [ ]:
import os
import json
import tkinter as tk
from tkinter import ttk, scrolledtext, messagebox, filedialog


class IslamicFinanceAccountingApp:
    """GUI application for Islamic Finance Accounting Agent."""
    
    def __init__(self, root):
        self.root = root
        self.root.title("Islamic Finance Accounting Agent")
        self.root.geometry("1200x800")
        
        # Initialize the accounting system
        self.system = IslamicFinanceAccountingSystem()
        
        # Set up the main frame
        self.setup_ui()
        
        # Load existing use cases
        self.load_use_cases()
    
    def setup_ui(self):
        """Set up the user interface."""
        # Create notebook (tabbed interface)
        self.notebook = ttk.Notebook(self.root)
        self.notebook.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
        
        # Create tabs
        self.query_tab = ttk.Frame(self.notebook)
        self.use_cases_tab = ttk.Frame(self.notebook)
        self.add_use_case_tab = ttk.Frame(self.notebook)
        
        self.notebook.add(self.query_tab, text="Query Processor")
        self.notebook.add(self.use_cases_tab, text="Use Cases")
        self.notebook.add(self.add_use_case_tab, text="Add Use Case")
        
        # Set up each tab
        self.setup_query_tab()
        self.setup_use_cases_tab()
        self.setup_add_use_case_tab()
    
    def setup_query_tab(self):
        """Set up the query processing tab."""
        # Query input frame
        input_frame = ttk.LabelFrame(self.query_tab, text="Enter Your Query")
        input_frame.pack(fill=tk.BOTH, expand=False, padx=10, pady=10)
        
        self.query_input = scrolledtext.ScrolledText(input_frame, wrap=tk.WORD, height=8)
        self.query_input.pack(fill=tk.BOTH, expand=True, padx=5, pady=5)
        
        # Sample query button
        sample_button = ttk.Button(input_frame, text="Load Sample Query", command=self.load_sample_query)
        sample_button.pack(side=tk.LEFT, padx=5, pady=5)
        
        # Process button
        process_button = ttk.Button(input_frame, text="Process Query", command=self.process_query)
        process_button.pack(side=tk.RIGHT, padx=5, pady=5)
        
        # Results frame
        results_frame = ttk.LabelFrame(self.query_tab, text="Results")
        results_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
        
        # Split the results frame into two panels
        self.results_paned = ttk.PanedWindow(results_frame, orient=tk.HORIZONTAL)
        self.results_paned.pack(fill=tk.BOTH, expand=True, padx=5, pady=5)
        
        # Generated solution frame
        gen_solution_frame = ttk.LabelFrame(self.results_paned, text="Generated Solution")
        self.generated_solution = scrolledtext.ScrolledText(gen_solution_frame, wrap=tk.WORD)
        self.generated_solution.pack(fill=tk.BOTH, expand=True, padx=5, pady=5)
        
        # Correct solution frame
        correct_solution_frame = ttk.LabelFrame(self.results_paned, text="Correct Solution")
        self.correct_solution = scrolledtext.ScrolledText(correct_solution_frame, wrap=tk.WORD)
        self.correct_solution.pack(fill=tk.BOTH, expand=True, padx=5, pady=5)
        
        # Add both frames to the paned window
        self.results_paned.add(gen_solution_frame, weight=1)
        self.results_paned.add(correct_solution_frame, weight=1)
    
    def setup_use_cases_tab(self):
        """Set up the use cases tab."""
        # Filter frame
        filter_frame = ttk.Frame(self.use_cases_tab)
        filter_frame.pack(fill=tk.X, padx=10, pady=5)
        
        ttk.Label(filter_frame, text="Filter by Category:").pack(side=tk.LEFT, padx=5)
        self.category_filter = ttk.Combobox(filter_frame, width=30)
        self.category_filter.pack(side=tk.LEFT, padx=5)
        self.category_filter.bind("<<ComboboxSelected>>", self.filter_use_cases)
        
        ttk.Button(filter_frame, text="Export Use Cases", command=self.export_use_cases).pack(side=tk.RIGHT, padx=5)
        ttk.Button(filter_frame, text="Import Use Cases", command=self.import_use_cases).pack(side=tk.RIGHT, padx=5)
        
        # Use cases table
        table_frame = ttk.Frame(self.use_cases_tab)
        table_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=5)
        
        # Create treeview
        self.use_cases_tree = ttk.Treeview(
            table_frame, 
            columns=("category", "subcategory", "scenario", "question"),
            show="headings"
        )
        
        # Set column headings
        self.use_cases_tree.heading("category", text="Category")
        self.use_cases_tree.heading("subcategory", text="Subcategory")
        self.use_cases_tree.heading("scenario", text="Scenario")
        self.use_cases_tree.heading("question", text="Question")
        
        # Set column widths
        self.use_cases_tree.column("category", width=150)
        self.use_cases_tree.column("subcategory", width=150)
        self.use_cases_tree.column("scenario", width=300)
        self.use_cases_tree.column("question", width=300)
        
        # Add scrollbars
        vsb = ttk.Scrollbar(table_frame, orient="vertical", command=self.use_cases_tree.yview)
        hsb = ttk.Scrollbar(table_frame, orient="horizontal", command=self.use_cases_tree.xview)
        self.use_cases_tree.configure(yscrollcommand=vsb.set, xscrollcommand=hsb.set)
        
        # Grid layout
        self.use_cases_tree.grid(column=0, row=0, sticky='nsew')
        vsb.grid(column=1, row=0, sticky='ns')
        hsb.grid(column=0, row=1, sticky='ew')
        
        # Configure the grid
        table_frame.grid_columnconfigure(0, weight=1)
        table_frame.grid_rowconfigure(0, weight=1)
        
        # Bind double-click event to view use case details
        self.use_cases_tree.bind("<Double-1>", self.view_use_case_details)
    
    def setup_add_use_case_tab(self):
        """Set up the tab for adding new use cases."""
        # Form frame
        form_frame = ttk.Frame(self.add_use_case_tab)
        form_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
        
        # Create a notebook for the form sections
        form_notebook = ttk.Notebook(form_frame)
        form_notebook.pack(fill=tk.BOTH, expand=True)
        
        # Basic info tab
        basic_info_tab = ttk.Frame(form_notebook)
        form_notebook.add(basic_info_tab, text="Basic Information")
        
        # Scenario tab
        scenario_tab = ttk.Frame(form_notebook)
        form_notebook.add(scenario_tab, text="Scenario & Question")
        
        # Solution tab
        solution_tab = ttk.Frame(form_notebook)
        form_notebook.add(solution_tab, text="Solution & Metadata")
        
        # Setup each tab
        self.setup_basic_info_tab(basic_info_tab)
        self.setup_scenario_tab(scenario_tab)
        self.setup_solution_tab(solution_tab)
        
        # Buttons frame
        button_frame = ttk.Frame(self.add_use_case_tab)
        button_frame.pack(fill=tk.X, padx=10, pady=5)
        
        ttk.Button(button_frame, text="Clear Form", command=self.clear_form).pack(side=tk.LEFT, padx=5)
        ttk.Button(button_frame, text="Add Use Case", command=self.add_new_use_case).pack(side=tk.RIGHT, padx=5)
    
    def setup_basic_info_tab(self, parent):
        """Set up the basic information form tab."""
        # Add fields with labels
        ttk.Label(parent, text="Category:").grid(row=0, column=0, sticky=tk.W, padx=5, pady=5)
        self.category_entry = ttk.Combobox(parent, width=40)
        self.category_entry['values'] = ["Ijarah MBT Accounting", "Murabaha Accounting", "Sukuk Accounting", "Takaful Accounting"]
        self.category_entry.grid(row=0, column=1, sticky=tk.W, padx=5, pady=5)
        
        ttk.Label(parent, text="Subcategory:").grid(row=1, column=0, sticky=tk.W, padx=5, pady=5)
        self.subcategory_entry = ttk.Combobox(parent, width=40)
        self.subcategory_entry['values'] = ["Lessee's Books", "Lessor's Books", "Buyer's Books", "Seller's Books"]
        self.subcategory_entry.grid(row=1, column=1, sticky=tk.W, padx=5, pady=5)
        
        ttk.Label(parent, text="Related Standard:").grid(row=2, column=0, sticky=tk.W, padx=5, pady=5)
        self.standard_entry = ttk.Combobox(parent, width=40)
        self.standard_entry['values'] = ["AAOIFI FAS 8", "AAOIFI FAS 7", "IFRS 16", "AAOIFI FAS 30"]
        self.standard_entry.grid(row=2, column=1, sticky=tk.W, padx=5, pady=5)
    
    def setup_scenario_tab(self, parent):
        """Set up the scenario and question form tab."""
        ttk.Label(parent, text="Scenario:").grid(row=0, column=0, sticky=tk.NW, padx=5, pady=5)
        self.scenario_text = scrolledtext.ScrolledText(parent, wrap=tk.WORD, height=10)
        self.scenario_text.grid(row=0, column=1, sticky=tk.NSEW, padx=5, pady=5)
        
        ttk.Label(parent, text="Question:").grid(row=1, column=0, sticky=tk.NW, padx=5, pady=5)
        self.question_text = scrolledtext.ScrolledText(parent, wrap=tk.WORD, height=6)
        self.question_text.grid(row=1, column=1, sticky=tk.NSEW, padx=5, pady=5)
        
        # Configure grid to expand with window
        parent.grid_columnconfigure(1, weight=1)
        parent.grid_rowconfigure(0, weight=2)
        parent.grid_rowconfigure(1, weight=1)
    
    def setup_solution_tab(self, parent):
        """Set up the solution and metadata form tab."""
        ttk.Label(parent, text="Correct Solution:").grid(row=0, column=0, sticky=tk.NW, padx=5, pady=5)
        self.solution_text = scrolledtext.ScrolledText(parent, wrap=tk.WORD, height=10)
        self.solution_text.grid(row=0, column=1, sticky=tk.NSEW, padx=5, pady=5)
        
        # Metadata Frame
        metadata_frame = ttk.LabelFrame(parent, text="Metadata")
        metadata_frame.grid(row=1, column=0, columnspan=2, sticky=tk.NSEW, padx=5, pady=5)
        
        # Add metadata fields
        ttk.Label(metadata_frame, text="Asset Cost:").grid(row=0, column=0, sticky=tk.W, padx=5, pady=2)
        self.asset_cost_entry = ttk.Entry(metadata_frame, width=15)
        self.asset_cost_entry.grid(row=0, column=1, sticky=tk.W, padx=5, pady=2)
        
        ttk.Label(metadata_frame, text="Import Tax:").grid(row=1, column=0, sticky=tk.W, padx=5, pady=2)
        self.import_tax_entry = ttk.Entry(metadata_frame, width=15)
        self.import_tax_entry.grid(row=1, column=1, sticky=tk.W, padx=5, pady=2)
        
        ttk.Label(metadata_frame, text="Freight:").grid(row=2, column=0, sticky=tk.W, padx=5, pady=2)
        self.freight_entry = ttk.Entry(metadata_frame, width=15)
        self.freight_entry.grid(row=2, column=1, sticky=tk.W, padx=5, pady=2)
        
        ttk.Label(metadata_frame, text="Ijarah Term (years):").grid(row=0, column=2, sticky=tk.W, padx=5, pady=2)
        self.ijarah_term_entry = ttk.Entry(metadata_frame, width=15)
        self.ijarah_term_entry.grid(row=0, column=3, sticky=tk.W, padx=5, pady=2)
        
        ttk.Label(metadata_frame, text="Residual Value:").grid(row=1, column=2, sticky=tk.W, padx=5, pady=2)
        self.residual_value_entry = ttk.Entry(metadata_frame, width=15)
        self.residual_value_entry.grid(row=1, column=3, sticky=tk.W, padx=5, pady=2)
        
        ttk.Label(metadata_frame, text="Purchase Price:").grid(row=2, column=2, sticky=tk.W, padx=5, pady=2)
        self.purchase_price_entry = ttk.Entry(metadata_frame, width=15)
        self.purchase_price_entry.grid(row=2, column=3, sticky=tk.W, padx=5, pady=2)
        
        ttk.Label(metadata_frame, text="Annual Rental:").grid(row=3, column=0, sticky=tk.W, padx=5, pady=2)
        self.annual_rental_entry = ttk.Entry(metadata_frame, width=15)
        self.annual_rental_entry.grid(row=3, column=1, sticky=tk.W, padx=5, pady=2)
        
        # Configure grid to expand with window
        parent.grid_columnconfigure(1, weight=1)
        parent.grid_rowconfigure(0, weight=1)
    
    def view_use_case_details(self, event=None):
        """View details of the selected use case in a pop-up window."""
        # Get selected item
        selection = self.use_cases_tree.selection()
        if not selection:
            return
            
        item_id = selection[0]
        item_index = int(item_id)
        
        # Get the use case
        use_case = self.system.ijarah_agent.use_cases[item_index]
        
        # Create a pop-up window
        details_window = tk.Toplevel(self.root)
        details_window.title(f"Use Case Details: {use_case.category} - {use_case.subcategory}")
        details_window.geometry("800x600")
        
        # Create a notebook for organized display
        notebook = ttk.Notebook(details_window)
        notebook.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
        
        # Scenario tab
        scenario_tab = ttk.Frame(notebook)
        notebook.add(scenario_tab, text="Scenario & Question")
        
        ttk.Label(scenario_tab, text="Scenario:").grid(row=0, column=0, sticky=tk.NW, padx=5, pady=5)
        scenario_text = scrolledtext.ScrolledText(scenario_tab, wrap=tk.WORD, height=8)
        scenario_text.grid(row=0, column=1, sticky=tk.NSEW, padx=5, pady=5)
        scenario_text.insert(tk.END, use_case.scenario)
        scenario_text.configure(state="disabled")
        
        ttk.Label(scenario_tab, text="Question:").grid(row=1, column=0, sticky=tk.NW, padx=5, pady=5)
        question_text = scrolledtext.ScrolledText(scenario_tab, wrap=tk.WORD, height=4)
        question_text.grid(row=1, column=1, sticky=tk.NSEW, padx=5, pady=5)
        question_text.insert(tk.END, use_case.question)
        question_text.configure(state="disabled")
        
        # Solution tab
        solution_tab = ttk.Frame(notebook)
        notebook.add(solution_tab, text="Solution")
        
        solution_text = scrolledtext.ScrolledText(solution_tab, wrap=tk.WORD)
        solution_text.pack(fill=tk.BOTH, expand=True, padx=5, pady=5)
        solution_text.insert(tk.END, use_case.correct_solution)
        solution_text.configure(state="disabled")
        
        # Metadata tab
        metadata_tab = ttk.Frame(notebook)
        notebook.add(metadata_tab, text="Metadata")
        
        metadata_text = scrolledtext.ScrolledText(metadata_tab, wrap=tk.WORD)
        metadata_text.pack(fill=tk.BOTH, expand=True, padx=5, pady=5)
        
        metadata_str = "Category: " + use_case.category + "\n"
        metadata_str += "Subcategory: " + use_case.subcategory + "\n\n"
        metadata_str += "Standard: " + use_case.metadata.get("standard", "Not specified") + "\n\n"
        
        # Add numeric metadata
        for key, value in use_case.metadata.items():
            if key != "standard" and isinstance(value, (int, float)):
                metadata_str += f"{key.replace('_', ' ').title()}: {value:,.2f}\n"
        
        metadata_text.insert(tk.END, metadata_str)
        metadata_text.configure(state="disabled")
        
        # Configure grid layout
        scenario_tab.grid_columnconfigure(1, weight=1)
        scenario_tab.grid_rowconfigure(0, weight=2)
        scenario_tab.grid_rowconfigure(1, weight=1)
    
    def load_sample_query(self):
        """Load a sample query into the input box."""
        sample_query = """I need the initial recognition entry for an Ijarah MBT arrangement where the lessee (Islamic bank)
is leasing a generator purchased at USD 450,000 with USD 12,000 import tax and USD 30,000 freight.
The Ijarah term is 2 years with yearly rental of USD 300,000 and expected residual value of USD 5,000.
The lessee will purchase the asset at the end for USD 3,000."""
        
        self.query_input.delete(1.0, tk.END)
        self.query_input.insert(tk.END, sample_query)
    
    def process_query(self):
        """Process the user's query and display results."""
        query = self.query_input.get(1.0, tk.END)
        if not query.strip():
            messagebox.showwarning("Empty Query", "Please enter a query before processing.")
            return
            
        try:
            # Process the query using the Islamic Finance Accounting System
            result = self.system.process_query(query)
            
            # Clear previous results
            self.generated_solution.delete(1.0, tk.END)
            self.correct_solution.delete(1.0, tk.END)
            
            # Display the generated solution
            if "formatted_solution" in result:
                self.generated_solution.insert(tk.END, result["formatted_solution"])
                
            # Display the correct solution if available
            if "correct_solution" in result:
                self.correct_solution.insert(tk.END, result["correct_solution"])
                
            # If no matching case was found, show notification
            if result.get("found_matching_case", False) == False:
                messagebox.showinfo("No Exact Match", 
                                  "No exact matching use case was found. Generated solution is based on query parsing.")
                
        except Exception as e:
            messagebox.showerror("Error", f"An error occurred while processing your query: {str(e)}")
    
    def add_new_use_case(self):
        """Add a new use case to the system from the form data."""
        try:
            # Collect form data
            category = self.category_entry.get()
            subcategory = self.subcategory_entry.get()
            scenario = self.scenario_text.get(1.0, tk.END).strip()
            question = self.question_text.get(1.0, tk.END).strip()
            correct_solution = self.solution_text.get(1.0, tk.END).strip()
            standard = self.standard_entry.get()
            
            # Validate required fields
            if not all([category, subcategory, scenario, question, correct_solution]):
                messagebox.showwarning("Missing Information", 
                                     "Please fill in all required fields (Category, Subcategory, Scenario, Question, and Solution).")
                return
            
            # Collect metadata
            metadata = {}
            
            # Try to convert numeric fields
            try:
                if self.asset_cost_entry.get():
                    metadata["asset_cost"] = float(self.asset_cost_entry.get())
                if self.import_tax_entry.get():
                    metadata["import_tax"] = float(self.import_tax_entry.get())
                if self.freight_entry.get():
                    metadata["freight"] = float(self.freight_entry.get())
                if self.ijarah_term_entry.get():
                    metadata["ijarah_term"] = int(self.ijarah_term_entry.get())
                if self.residual_value_entry.get():
                    metadata["residual_value"] = float(self.residual_value_entry.get())
                if self.purchase_price_entry.get():
                    metadata["purchase_price"] = float(self.purchase_price_entry.get())
                if self.annual_rental_entry.get():
                    metadata["annual_rental"] = float(self.annual_rental_entry.get())
            except ValueError:
                messagebox.showwarning("Invalid Input", 
                                     "Please ensure all numeric fields contain valid numbers.")
                return
                
            # Add standard to metadata
            if standard:
                metadata["standard"] = standard
                
            # Create the new use case
            new_use_case = AccountingUseCase(
                category=category,
                subcategory=subcategory,
                scenario=scenario,
                question=question,
                correct_solution=correct_solution,
                metadata=metadata
            )
            
            # Add to the system
            self.system.ijarah_agent.add_use_case(new_use_case, standard if standard else None)
            
            # Update the use cases display
            self.refresh_use_cases_display()
            
            # Show success message
            messagebox.showinfo("Success", "New use case added successfully.")
            
            # Clear the form
            self.clear_form()
            
        except Exception as e:
            messagebox.showerror("Error", f"An error occurred while adding the use case: {str(e)}")
    
    def clear_form(self):
        """Clear all form fields in the add use case tab."""
        self.category_entry.set("")
        self.subcategory_entry.set("")
        self.standard_entry.set("")
        self.scenario_text.delete(1.0, tk.END)
        self.question_text.delete(1.0, tk.END)
        self.solution_text.delete(1.0, tk.END)
        self.asset_cost_entry.delete(0, tk.END)
        self.import_tax_entry.delete(0, tk.END)
        self.freight_entry.delete(0, tk.END)
        self.ijarah_term_entry.delete(0, tk.END)
        self.residual_value_entry.delete(0, tk.END)
        self.purchase_price_entry.delete(0, tk.END)
        self.annual_rental_entry.delete(0, tk.END)
    
    def load_use_cases(self):
        """Load existing use cases and populate the UI."""
        # Get all use cases
        use_cases = self.system.ijarah_agent.use_cases
        
        # Set up the display
        self.refresh_use_cases_display()
        
        # Populate the category filter dropdown
        categories = sorted(set(uc.category for uc in use_cases))
        self.category_filter['values'] = ["All Categories"] + categories
        self.category_filter.current(0)  # Default to "All Categories"
    
    def refresh_use_cases_display(self):
        """Refresh the display of use cases in the treeview."""
        # Clear existing items
        for item in self.use_cases_tree.get_children():
            self.use_cases_tree.delete(item)
            
        # Get filtered use cases
        selected_category = self.category_filter.get()
        use_cases = self.system.ijarah_agent.use_cases
        
        if selected_category and selected_category != "All Categories":
            use_cases = [uc for uc in use_cases if uc.category == selected_category]
            
        # Add use cases to the treeview
        for i, uc in enumerate(use_cases):
            # Truncate long fields for display
            scenario_preview = uc.scenario[:50] + "..." if len(uc.scenario) > 50 else uc.scenario
            question_preview = uc.question[:50] + "..." if len(uc.question) > 50 else uc.question
            
            self.use_cases_tree.insert("", "end", iid=i, text=str(i+1),
                                     values=(uc.category, uc.subcategory, 
                                            scenario_preview, question_preview))
    
    def filter_use_cases(self, event=None):
        """Filter use cases based on the selected category."""
        self.refresh_use_cases_display()
    
    def export_use_cases(self):
        """Export use cases to a JSON file."""
        file_path = filedialog.asksaveasfilename(
            defaultextension=".json",
            filetypes=[("JSON files", "*.json"), ("All files", "*.*")],
            title="Export Use Cases"
        )
        
        if not file_path:
            return  # User cancelled
            
        try:
            success = self.system.ijarah_agent.save_use_cases_to_json(file_path)
            if success:
                messagebox.showinfo("Export Successful", 
                                  f"Successfully exported {len(self.system.ijarah_agent.use_cases)} use cases to {file_path}")
            else:
                messagebox.showerror("Export Failed", 
                                   "Failed to export use cases. Please check console for details.")
        except Exception as e:
            messagebox.showerror("Export Error", 
                               f"An error occurred during export: {str(e)}")
    
    def import_use_cases(self):
        """Import use cases from a JSON file."""
        file_path = filedialog.askopenfilename(
            filetypes=[("JSON files", "*.json"), ("All files", "*.*")],
            title="Import Use Cases"
        )
        
        if not file_path:
            return  # User cancelled
            
        try:
            # Store current count for comparison
            old_count = len(self.system.ijarah_agent.use_cases)
            
            # Import use cases
            success = self.system.ijarah_agent.load_use_cases_from_json(file_path)
            
            if success:
                # Refresh display
                self.load_use_cases()
                
                # Calculate how many new use cases were added
                new_count = len(self.system.ijarah_agent.use_cases)
                added = new_count - old_count
                
                messagebox.showinfo("Import Successful", 
                                  f"Successfully imported {added} new use cases.")
            else:
                messagebox.showerror("Import Failed", 
                                   "Failed to import use cases. Please check console for details.")
        except Exception as e:
            messagebox.showerror("Import Error", 
                               f"An error occurred during import: {str(e)}")


def main():
    """Main function to start the application."""
    root = tk.Tk()
    app = IslamicFinanceAccountingApp(root)
    root.mainloop()


if __name__ == "__main__":
    main()

IndentationError: unindent does not match any outer indentation level (<string>, line 35)